# Data Extraction Module for Pricing & Offers System


## Imports


In [1]:
import os
import numpy as np
import pandas as pd
import snowflake.connector
import setup_environment_2


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (22.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


## Variables


In [2]:
# Region
REGION = "Egypt"

# Snowflake Warehouse
WAREHOUSE = "COMPUTE_WH"

# Date Variables
from datetime import datetime, timedelta
TODAY = datetime.now().date()
YESTERDAY = TODAY - timedelta(days=1)


## Initialize Environment


In [3]:
setup_environment_2.initialize_env()


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


## Warehouse & Cohort Mapping


In [4]:
# Warehouse Mapping: (region, warehouse_name, warehouse_id, cohort_id)
WAREHOUSE_MAPPING = [
    ('Cairo', 'Mostorod', 1, 700),
    ('Giza', 'Barageel', 236, 701),
    ('Giza', 'Sakkarah', 962, 701),
    ('Delta West', 'El-Mahala', 337, 703),
    ('Delta West', 'Tanta', 8, 703),
    ('Delta East', 'Mansoura FC', 339, 704),
    ('Delta East', 'Sharqya', 170, 704),
    ('Upper Egypt', 'Assiut FC', 501, 1124),
    ('Upper Egypt', 'Bani sweif', 401, 1126),
    ('Upper Egypt', 'Menya Samalot', 703, 1123),
    ('Upper Egypt', 'Sohag', 632, 1125),
    ('Alexandria', 'Khorshed Alex', 797, 702),
]

# Region to Cohort Mapping
REGION_COHORT_MAPPING = {
    'Cairo': 700,
    'Giza': 701,
    'Delta West': 703,
    'Delta East': 704,
    'Upper Egypt': 1124,
    'Alexandria': 702,
}

# All Cohort IDs
COHORT_IDS = [700, 701, 702, 703, 704, 1123, 1124, 1125, 1126]


## Snowflake Query Function


In [5]:
def query_snowflake(query):
    """Execute a query on Snowflake and return results as DataFrame."""
    con = snowflake.connector.connect(
        user=os.environ["SNOWFLAKE_USERNAME"],
        account=os.environ["SNOWFLAKE_ACCOUNT"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        database=os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        data = cur.fetchall()
        columns = [desc[0].lower() for desc in cur.description]  # Get column names from cursor
        return pd.DataFrame(data, columns=columns)
    except Exception as e:
        print(f"Snowflake Error: {e}")
        return pd.DataFrame()
    finally:
        cur.close()
        con.close()


In [6]:
def get_snowflake_timezone():
    """Get the current timezone from Snowflake."""
    query = "SHOW PARAMETERS LIKE 'TIMEZONE'"
    result = query_snowflake(query)
    return result.value[0] if len(result) > 0 else "UTC"


## Helper Functions


In [7]:
def get_warehouse_df():
    """Get warehouse mapping as DataFrame."""
    return pd.DataFrame(
        WAREHOUSE_MAPPING,
        columns=['region', 'warehouse', 'warehouse_id', 'cohort_id']
    )


def get_cohort_by_region(region):
    """Get cohort ID for a given region."""
    return REGION_COHORT_MAPPING.get(region)


def convert_to_numeric(df):
    """Convert DataFrame columns to numeric where possible."""
    df.columns = df.columns.str.lower()
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df


## Get Snowflake Timezone


In [8]:
TIMEZONE = get_snowflake_timezone()
print(f"Snowflake timezone: {TIMEZONE}")


Snowflake timezone: America/Los_Angeles


## Market Prices Extraction Queries
Queries for external market price data:
1. **Ben Soliman Prices** - Competitor reference prices
2. **Marketplace Prices** - Min, Max, Mod prices from marketplace
3. **Scrapped Data** - Competitor prices from scraping


In [9]:
# =============================================================================
# 1. BEN SOLIMAN PRICES QUERY
# =============================================================================
BEN_SOLIMAN_QUERY = f'''
WITH lower as (
    select distinct product_id, sku, new_d*bs_price as ben_soliman_price, INJECTION_DATE
    from (
        select maxab_product_id as product_id, maxab_sku as sku, INJECTION_DATE, wac1, wac_p,
            (bs_price/bs_unit_count) as bs_price, diff, cu_price,
            case when p1 > 1 then child_quantity else 0 end as scheck,
            round(p1/2)*2 as p1, p2,
            case when (ROUND(p1 / scheck) * scheck) = 0 then p1 else (ROUND(p1 / scheck) * scheck) end as new_d
        from (
            select sm.*, wac1, wac_p, 
                abs((bs_price/bs_unit_count)-(wac_p*maxab_basic_unit_count))/(wac_p*maxab_basic_unit_count) as diff,
                cpc.price as cu_price, pup.child_quantity,
                round((cu_price/(bs_price/bs_unit_count))) as p1, 
                round(((bs_price/bs_unit_count)/cu_price)) as p2
            from materialized_views.savvy_mapping sm 
            join finance.all_cogs f on f.product_id = sm.maxab_product_id 
                and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_Date and f.to_date
            join PACKING_UNIT_PRODUCTS pu on pu.product_id = sm.maxab_product_id and pu.IS_BASIC_UNIT = 1 
            join cohort_product_packing_units cpc on cpc.PRODUCT_PACKING_UNIT_ID = pu.id and cohort_id = 700 
            join packing_unit_products pup on pup.product_id = sm.maxab_product_id and pup.is_basic_unit = 1  
            where bs_price is not null 
                and INJECTION_DATE::date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 5 
                and diff > 0.3 and p1 > 1
        )
    )
    qualify max(INJECTION_DATE) over(partition by product_id) = INJECTION_DATE
),

m_bs as (
    select z.* from (
        select maxab_product_id as product_id, maxab_sku as sku, avg(bs_final_price) as ben_soliman_price, INJECTION_DATE
        from (
            select *, row_number() over(partition by maxab_product_id order by diff) as rnk_2 
            from (
                select *, (bs_final_price-wac_p)/wac_p as diff_2 
                from (
                    select *, bs_price/maxab_basic_unit_count as bs_final_price 
                    from (
                        select *, row_number() over(partition by maxab_product_id, maxab_pu order by diff) as rnk 
                        from (
                            select *, max(INJECTION_DATE::date) over(partition by maxab_product_id, maxab_pu) as max_date
                            from (
                                select sm.*, wac1, wac_p, 
                                    abs(bs_price-(wac_p*maxab_basic_unit_count))/(wac_p*maxab_basic_unit_count) as diff 
                                from materialized_views.savvy_mapping sm 
                                join finance.all_cogs f on f.product_id = sm.maxab_product_id 
                                    and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_Date and f.to_date
                                where bs_price is not null 
                                    and INJECTION_DATE::date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 5 
                                    and diff < 0.3
                            )
                            qualify max_date = INJECTION_DATE
                        ) qualify rnk = 1 
                    )
                ) where diff_2 between -0.5 and 0.5 
            ) qualify rnk_2 = 1 
        ) group by all
    ) z 
    join finance.all_cogs f on f.product_id = z.product_id 
        and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_Date and f.to_date
    where ben_soliman_price between f.wac_p*0.8 and f.wac_p*1.3
)

select product_id, avg(ben_soliman_price) as ben_soliman_price
from (
    select * from (
        select * from m_bs 
        union all
        select * from lower
    )
    qualify max(INJECTION_DATE) over(partition by product_id) = INJECTION_DATE
)
group by all
'''


In [10]:
# =============================================================================
# 2. MARKETPLACE PRICES QUERY (with region fallback)
# =============================================================================
MARKETPLACE_PRICES_QUERY = f'''
WITH MP as (
    select region, product_id,
        min(min_price) as min_price, min(max_price) as max_price,
        min(mod_price) as mod_price, min(true_min) as true_min, min(true_max) as true_max
    from (
        select mp.region, mp.product_id, mp.pu_id,
            min_price/BASIC_UNIT_COUNT as min_price,
            max_price/BASIC_UNIT_COUNT as max_price,
            mod_price/BASIC_UNIT_COUNT as mod_price,
            TRUE_MIN_PRICE/BASIC_UNIT_COUNT as true_min,
            TRUE_MAX_PRICE/BASIC_UNIT_COUNT as true_max
        from materialized_views.marketplace_prices mp 
        join packing_unit_products pup on pup.product_id = mp.product_id and pup.packing_unit_id = mp.pu_id
        join finance.all_cogs f on f.product_id = mp.product_id 
            and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_date and f.to_date
        where least(min_price, mod_price) between wac_p*0.9 and wac_p*1.3 
    )
    group by all 
),

region_mapping AS (
    SELECT * FROM (VALUES
        ('Delta East', 'Delta West'), ('Delta West', 'Delta East'),
        ('Alexandria', 'Cairo'), ('Alexandria', 'Giza'),
        ('Upper Egypt', 'Cairo'), ('Upper Egypt', 'Giza'),
        ('Cairo', 'Giza'), ('Giza', 'Cairo'),
        ('Delta West', 'Cairo'), ('Delta East', 'Cairo'),
        ('Delta West', 'Giza'), ('Delta East', 'Giza')
    ) AS region_mapping(region, fallback_region)
),

all_regions as (
    SELECT * FROM (VALUES
        ('Cairo'), ('Giza'), ('Delta West'), ('Delta East'), ('Upper Egypt'), ('Alexandria')
    ) AS x(region)
),

full_data as (
    select products.id as product_id, ar.region
    from products, all_regions ar
    where activation = 'true'
)

select region, product_id,
    min(final_min_price) as final_min_price, 
    min(final_max_price) as final_max_price,
    min(final_mod_price) as final_mod_price, 
    min(final_true_min) as final_true_min,
    min(final_true_max) as final_true_max
from (
    SELECT distinct w.region, w.product_id,
        COALESCE(m1.min_price, m2.min_price) AS final_min_price,
        COALESCE(m1.max_price, m2.max_price) AS final_max_price,
        COALESCE(m1.mod_price, m2.mod_price) AS final_mod_price,
        COALESCE(m1.true_min, m2.true_min) AS final_true_min,
        COALESCE(m1.true_max, m2.true_max) AS final_true_max
    FROM full_data w
    LEFT JOIN MP m1 ON w.region = m1.region and w.product_id = m1.product_id
    LEFT JOIN region_mapping rm ON w.region = rm.region
    LEFT JOIN MP m2 ON rm.fallback_region = m2.region AND w.product_id = m2.product_id
)
where final_min_price is not null 
group by all
'''


In [11]:
# =============================================================================
# 3. SCRAPPED DATA QUERY (Competitor prices from scraping)
# =============================================================================
SCRAPPED_DATA_QUERY = f'''
select product_id, region,
    MIN(market_price) AS min_scrapped,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY market_price) AS scrapped25,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY market_price) AS scrapped50,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY market_price) AS scrapped75,
    MAX(market_price) AS max_scrapped
from (
    select distinct cmp.*, max(date) over(partition by region, cmp.product_id, competitor) as max_date
    from MATERIALIZED_VIEWS.CLEANED_MARKET_PRICES cmp
    join finance.all_cogs f on f.product_id = cmp.product_id 
        and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) between f.from_date and f.to_date 
    where date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 7 
        and MARKET_PRICE between f.wac_p * 0.8 and wac_p * 1.3
    qualify date = max_date 
)
group by all
'''


In [12]:
## Additional Data Queries (Sales, Groups, WAC)


In [13]:
# =============================================================================
# 4. PRODUCT BASE DATA QUERY (product_id, sku, brand, cat, wac1, wac_p, current_price)
# =============================================================================
PRODUCT_BASE_QUERY = f'''
WITH skus_prices AS (
    WITH local_prices AS (
        SELECT  
            CASE 
                WHEN cpu.cohort_id IN (700, 695) THEN 'Cairo'
                WHEN cpu.cohort_id IN (701) THEN 'Giza'
                WHEN cpu.cohort_id IN (704, 698) THEN 'Delta East'
                WHEN cpu.cohort_id IN (703, 697) THEN 'Delta West'
                WHEN cpu.cohort_id IN (696, 1123, 1124, 1125, 1126) THEN 'Upper Egypt'
                WHEN cpu.cohort_id IN (702, 699) THEN 'Alexandria'
            END AS region,
            cohort_id,
            pu.product_id,
            pu.packing_unit_id,
            pu.basic_unit_count,
            AVG(cpu.price) AS price
        FROM cohort_product_packing_units cpu
        JOIN PACKING_UNIT_PRODUCTS pu ON pu.id = cpu.product_packing_unit_id
        WHERE cpu.cohort_id IN (700,701,702,703,704,695,696,697,698,699,1123,1124,1125,1126)
            AND cpu.created_at::date <> '2023-07-31'
            AND cpu.is_customized = TRUE
        GROUP BY ALL
    ),
    
    live_prices AS (
        SELECT 
            region, cohort_id, product_id, 
            pu_id AS packing_unit_id, 
            buc AS basic_unit_count, 
            NEW_PRICE AS price
        FROM materialized_views.DBDP_PRICES
        WHERE created_at = CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date
            AND DATE_PART('hour', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::time) 
                BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND (SPLIT_PART(time_slot, '-', 1)::int) + 1
            AND cohort_id IN (700,701,702,703,704,695,696,697,698,699,1123,1124,1125,1126)
    ),
    
    prices AS (
        SELECT *
        FROM (
            SELECT *, 1 AS priority FROM live_prices
            UNION ALL
            SELECT *, 2 AS priority FROM local_prices
        )
        QUALIFY ROW_NUMBER() OVER (PARTITION BY region, cohort_id, product_id, packing_unit_id ORDER BY priority) = 1
    )
    
    SELECT region, cohort_id, product_id, price
    FROM prices
    WHERE basic_unit_count = 1
        AND ((product_id = 1309 AND packing_unit_id = 2) OR (product_id <> 1309))
)

SELECT distinct
    region, cohort_id, p.product_id,
    CONCAT(products.name_ar, ' ', products.size, ' ', product_units.name_ar) AS sku,
    b.name_ar AS brand,
    cat.name_ar AS cat,
    wac1, wac_p, p.price as current_price
FROM skus_prices p
JOIN finance.all_cogs c ON c.product_id = p.product_id 
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) BETWEEN c.from_date AND c.to_date
JOIN products ON products.id = p.product_id
JOIN categories cat ON cat.id = products.category_id
JOIN brands b ON b.id = products.brand_id
JOIN product_units ON product_units.id = products.unit_id
WHERE wac1 > 0 AND wac_p > 0
GROUP BY ALL
'''

# =============================================================================
# 5. SALES DATA QUERY (120-day NMV by cohort/product)
# =============================================================================
SALES_QUERY = f'''
SELECT DISTINCT cpc.cohort_id, pso.product_id,
    CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
    brands.name_ar as brand, categories.name_ar as cat,
    sum(pso.total_price) as nmv
FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN COHORT_PRICING_CHANGES cpc ON cpc.id = pso.COHORT_PRICING_CHANGE_id
JOIN products ON products.id = pso.product_id
JOIN brands ON products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN product_units ON product_units.id = products.unit_id 
WHERE so.created_at::date BETWEEN CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 120 
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 1 
    AND so.sales_order_status_id NOT IN (7, 12)
    AND so.channel IN ('telesales', 'retailer')
    AND pso.purchased_item_count <> 0
    AND cpc.cohort_id IN (700,701,702,703,704,1123,1124,1125,1126)
GROUP BY ALL
'''

# =============================================================================
# 6. MARGIN STATS QUERY (STD and average margins)  
# =============================================================================
MARGIN_STATS_QUERY = f'''
select product_id, cohort_id, 
    (0.6*product_std) + (0.3*brand_std) + (0.1*cat_std) as std, 
    avg_margin
from (
    select product_id, cohort_id, 
        stddev(product_margin) as product_std, 
        stddev(brand_margin) as brand_std,
        stddev(cat_margin) as cat_std, 
        avg(product_margin) as avg_margin
    from (
        select distinct product_id, order_date, cohort_id,
            (nmv-cogs_p)/nmv as product_margin, 
            (brand_nmv-brand_cogs)/brand_nmv as brand_margin,
            (cat_nmv-cat_cogs)/cat_nmv as cat_margin
        from (
            SELECT DISTINCT so.created_at::date as order_date, cpc.cohort_id, pso.product_id,
                brands.name_ar as brand, categories.name_ar as cat,
                sum(COALESCE(f.wac_p,0) * pso.purchased_item_count * pso.basic_unit_count) as cogs_p,
                sum(pso.total_price) as nmv,
                sum(nmv) over(partition by order_date, cat, brand) as brand_nmv,
                sum(cogs_p) over(partition by order_date, cat, brand) as brand_cogs,
                sum(nmv) over(partition by order_date, cat) as cat_nmv,
                sum(cogs_p) over(partition by order_date, cat) as cat_cogs
            FROM product_sales_order pso
            JOIN sales_orders so ON so.id = pso.sales_order_id   
            JOIN COHORT_PRICING_CHANGES cpc on cpc.id = pso.cohort_pricing_change_id
            JOIN products on products.id = pso.product_id
            JOIN brands on products.brand_id = brands.id 
            JOIN categories ON products.category_id = categories.id
            JOIN finance.all_cogs f ON f.product_id = pso.product_id
                AND f.from_date::date <= so.created_at::date AND f.to_date::date > so.created_at::date
            WHERE so.created_at::date between 
                date_trunc('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - 120) 
                and CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date
                AND so.sales_order_status_id not in (7,12)
                AND so.channel IN ('telesales','retailer')
                AND pso.purchased_item_count <> 0
            GROUP BY ALL
        )
    ) group by all 
)
'''

# =============================================================================
# 7. TARGET MARGINS QUERY
# =============================================================================
TARGET_MARGINS_QUERY = f'''
WITH cat_brand_target as (
    SELECT DISTINCT cat, brand, margin as target_bm
    FROM performance.commercial_targets cplan
    QUALIFY CASE 
        WHEN DATE_TRUNC('month', MAX(DATE) OVER()) = DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date) 
        THEN DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date)
        ELSE DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - INTERVAL '1 month') 
    END = DATE_TRUNC('month', date)
),
cat_target as (
    select cat, sum(target_bm * (target_nmv/cat_total)) as cat_target_margin
    from (
        select *, sum(target_nmv) over(partition by cat) as cat_total
        from (
            select cat, brand, avg(target_bm) as target_bm, sum(target_nmv) as target_nmv
            from (
                SELECT DISTINCT date, city as region, cat, brand, margin as target_bm, nmv as target_nmv
                FROM performance.commercial_targets cplan
                QUALIFY CASE 
                    WHEN DATE_TRUNC('month', MAX(DATE) OVER()) = DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date) 
                    THEN DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date)
                    ELSE DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::date - INTERVAL '1 month') 
                END = DATE_TRUNC('month', date)
            ) group by all
        )
    ) group by all 
)
SELECT DISTINCT cbt.cat, cbt.brand, cbt.target_bm, ct.cat_target_margin
FROM cat_brand_target cbt
LEFT JOIN cat_target ct ON ct.cat = cbt.cat
'''


In [14]:
## Execute All Queries


In [15]:
# =============================================================================
# Execute all queries
# =============================================================================
print("Loading data from Snowflake...")

# 1. Ben Soliman Prices
print("  1. Loading Ben Soliman prices...")
df_ben_soliman = query_snowflake(BEN_SOLIMAN_QUERY)
df_ben_soliman = convert_to_numeric(df_ben_soliman)
print(f"     Loaded {len(df_ben_soliman)} Ben Soliman price records")

# 2. Marketplace Prices
print("  2. Loading marketplace prices...")
df_marketplace = query_snowflake(MARKETPLACE_PRICES_QUERY)
df_marketplace = convert_to_numeric(df_marketplace)
print(f"     Loaded {len(df_marketplace)} marketplace price records")

# 3. Scrapped Data
print("  3. Loading scrapped data...")
df_scrapped = query_snowflake(SCRAPPED_DATA_QUERY)
df_scrapped = convert_to_numeric(df_scrapped)
print(f"     Loaded {len(df_scrapped)} scrapped price records")

# 4. Product Base Data (product_id, sku, brand, cat, wac1, wac_p, current_price)
print("  4. Loading product base data...")
df_product_base = query_snowflake(PRODUCT_BASE_QUERY)
df_product_base = convert_to_numeric(df_product_base)
print(f"     Loaded {len(df_product_base)} product base records")

# 5. Sales Data
print("  5. Loading sales data...")
df_sales = query_snowflake(SALES_QUERY)
df_sales = convert_to_numeric(df_sales)
print(f"     Loaded {len(df_sales)} sales records")

# 6. Margin Stats
print("  6. Loading margin stats...")
df_margin_stats = query_snowflake(MARGIN_STATS_QUERY)
df_margin_stats = convert_to_numeric(df_margin_stats)
print(f"     Loaded {len(df_margin_stats)} margin stat records")

# 7. Target Margins
print("  7. Loading target margins...")
df_targets = query_snowflake(TARGET_MARGINS_QUERY)
df_targets = convert_to_numeric(df_targets)
print(f"     Loaded {len(df_targets)} target margin records")

# 8. Product Groups (from PostgreSQL)
print("  8. Loading product groups...")
df_groups = query_snowflake(
    '''SELECT * FROM materialized_views.sku_commercial_groups'''
)
df_groups.columns = df_groups.columns.str.lower()
df_groups = convert_to_numeric(df_groups)
print(f"     Loaded {len(df_groups)} group records")

print("\nAll queries completed!")
print(f"\n{'='*60}")
print("df_product_base DataFrame available with columns:")
print("  - region, cohort_id, product_id, sku, brand, cat, wac1, wac_p, current_price")
print(f"{'='*60}")


Loading data from Snowflake...
  1. Loading Ben Soliman prices...


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


     Loaded 1549 Ben Soliman price records
  2. Loading marketplace prices...


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


     Loaded 11209 marketplace price records
  3. Loading scrapped data...


...

/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


     Loaded 28930 margin stat records
  7. Loading target margins...


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


     Loaded 478 target margin records
  8. Loading product groups...
     Loaded 1649 group records

All queries completed!

df_product_base DataFrame available with columns:
  - region, cohort_id, product_id, sku, brand, cat, wac1, wac_p, current_price


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [16]:
# =============================================================================
# PART A: Build market_data DataFrame - Process market prices SEPARATELY
# =============================================================================
print("Building market_data DataFrame (market prices only)...")

# Create region-cohort mapping
REGION_COHORT_DF = pd.DataFrame({
    'region': ['Cairo', 'Giza', 'Delta West', 'Delta East', 
               'Upper Egypt', 'Upper Egypt', 'Upper Egypt', 'Upper Egypt', 'Alexandria'],
    'cohort_id': [700, 701, 703, 704, 1124, 1126, 1123, 1125, 702]
})

# =============================================================================
# Step 1: Outer join all market price sources
# =============================================================================
print("  Step 1: Joining all market price sources (outer join)...")

# Start with marketplace prices (has region + product_id)
market_data = df_marketplace.copy()

# Outer join with scrapped data (by region + product_id)
market_data = market_data.merge(df_scrapped, on=['region', 'product_id'], how='outer')

# Outer join with Ben Soliman prices (by product_id only - expand to all regions)
all_regions = pd.DataFrame({'region': ['Cairo', 'Giza', 'Delta West', 'Delta East', 'Upper Egypt', 'Alexandria']})
df_ben_soliman_expanded = df_ben_soliman.merge(all_regions, how='cross')

# Outer join with Ben Soliman
market_data = market_data.merge(df_ben_soliman_expanded, on=['region', 'product_id'], how='outer')

print(f"     Market prices base: {len(market_data)} records")

# =============================================================================
# Step 2: Add cohort_id and supporting data for market processing
# =============================================================================
print("  Step 2: Adding cohort IDs and supporting data for processing...")
market_data = market_data.merge(REGION_COHORT_DF, on='region')

# Need sales data for group processing (weighted median)
market_data = market_data.merge(
    df_sales[['cohort_id', 'product_id', 'nmv']], 
    on=['cohort_id', 'product_id'], 
    how='left'
)
market_data['nmv'] = market_data['nmv'].fillna(0)

# Need margin stats for price analysis
market_data = market_data.merge(df_margin_stats, on=['cohort_id', 'product_id'], how='left')

# Need WAC for price analysis - get from product base
market_data = market_data.merge(
    df_product_base[['cohort_id', 'product_id', 'wac_p', 'brand', 'cat']].drop_duplicates(), 
    on=['cohort_id', 'product_id'], 
    how='left'
)

# Need target margins for price analysis
market_data = market_data.merge(df_targets, on=['brand', 'cat'], how='left')
market_data['target_margin'] = market_data['target_bm'].fillna(market_data['cat_target_margin']).fillna(0)
market_data = market_data.drop(columns=['target_bm', 'cat_target_margin'], errors='ignore')

# Fill NaN values with defaults
market_data['std'] = market_data['std'].fillna(0.01)
market_data['avg_margin'] = market_data['avg_margin'].fillna(0)

# Merge product groups for group processing
market_data = market_data.merge(df_groups, on='product_id', how='left')

# Remove duplicates
market_data = market_data.drop_duplicates(subset=['cohort_id', 'product_id'])

# Filter out records without WAC (can't process prices without cost)
market_data = market_data[~market_data['wac_p'].isna()]

print(f"\n{'='*60}")
print(f"MARKET DATA BASE READY FOR PROCESSING")
print(f"{'='*60}")
print(f"Total records: {len(market_data)}")
print(f"  - With marketplace prices: {len(market_data[~market_data['final_min_price'].isna()])}")
print(f"  - With scrapped prices: {len(market_data[~market_data['min_scrapped'].isna()])}")
print(f"  - With Ben Soliman prices: {len(market_data[~market_data['ben_soliman_price'].isna()])}")


Building market_data DataFrame (market prices only)...
  Step 1: Joining all market price sources (outer join)...
     Market prices base: 16030 records
  Step 2: Adding cohort IDs and supporting data for processing...

MARKET DATA BASE READY FOR PROCESSING
Total records: 23674
  - With marketplace prices: 16462
  - With scrapped prices: 7661
  - With Ben Soliman prices: 13941


## PART A: Market Data Processing
Process market prices separately (group fill, coverage filter, price analysis, margin tiers)

In [17]:
# =============================================================================
# Group Processing - Calculate group-level aggregated prices (on market_data)
# =============================================================================

# Calculate group-level aggregated prices for products with group assignments
groups_data = market_data[~market_data['group_id'].isna()].copy()
groups_data['group_nmv'] = groups_data.groupby(['group_id', 'cohort_id'])['nmv'].transform('sum')
groups_data['cntrb'] = (groups_data['nmv'] / groups_data['group_nmv']).fillna(1)

# Flag if any price/scrapped column is non-NaN
price_cols = [
    'ben_soliman_price', 'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max',
    'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped'
]
groups_data['flag_non_nan'] = groups_data[price_cols].notna().any(axis=1).astype(int)

# Weighted Median Function
def weighted_median(series, weights):
    valid = ~series.isna() & ~weights.isna()
    s = series[valid]
    w = weights[valid]
    if len(s) == 0:
        return np.nan
    order = np.argsort(s)
    s, w = s.iloc[order], w.iloc[order]
    return s.iloc[np.searchsorted(np.cumsum(w), w.sum() / 2)]

# Perform Weighted Aggregation
groups_agg = (
    groups_data[groups_data['flag_non_nan'] == 1]
    .groupby(['group_id', 'cohort_id'])
    .apply(lambda g: pd.Series({
        col: weighted_median(g[col], g['cntrb']) for col in price_cols
    }))
    .reset_index()
)

# Fill missing prices with group-level prices
merged = market_data.merge(groups_agg, on=['group_id', 'cohort_id'], how='left', suffixes=('', '_group'))
for col in price_cols:
    merged[col] = merged[col].fillna(merged[f'{col}_group'])

market_data = merged.drop(columns=[f'{c}_group' for c in price_cols])

print(f"Market data after group processing: {len(market_data)} records")

Market data after group processing: 23674 records


/tmp/ipykernel_25983/2426940637.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


## Price Coverage Filtering

In [18]:
# =============================================================================
# Price Coverage Filtering - Filter products with sufficient price data (on market_data)
# =============================================================================

# Score price coverage
market_data['ben'] = 0
market_data['MP'] = 0
market_data['sp'] = 0

# Ben Soliman: 1 point if present
market_data.loc[~market_data['ben_soliman_price'].isna(), 'ben'] = 1

# Marketplace: 1 point if single price, 3 points if range
market_data.loc[(market_data['final_min_price'] == market_data['final_max_price']) & 
                (~market_data['final_min_price'].isna()), 'MP'] = 1
market_data.loc[(market_data['final_min_price'] != market_data['final_max_price']) & 
                (~market_data['final_min_price'].isna()), 'MP'] = 3

# Scrapped: 1 point if single price, 5 points if range
market_data.loc[(market_data['min_scrapped'] == market_data['max_scrapped']) & 
                (~market_data['min_scrapped'].isna()), 'sp'] = 1
market_data.loc[(market_data['min_scrapped'] != market_data['max_scrapped']) & 
                (~market_data['min_scrapped'].isna()), 'sp'] = 5

# Total price coverage score
market_data['total_p'] = market_data['ben'] + market_data['MP'] + market_data['sp']

# Filter: keep only products with total_p > 2
market_data = market_data[market_data['total_p'] > 2]

print(f"Market data after price coverage filtering: {len(market_data)} records")

Market data after price coverage filtering: 13075 records


## Price Analysis & Margin Calculation


In [19]:
# =============================================================================
# Price Analysis Functions
# =============================================================================

def price_analysis(row):
    """Analyze prices and calculate percentiles for a product."""
    wac = row['wac_p']
    avg_margin = row['avg_margin'] if row['avg_margin'] >= 0.01 else row['target_margin']
    std = np.maximum(row['std'], 0.0025)
    target_margin = row['target_margin']
    max_marg = np.maximum(avg_margin, target_margin)
    
    # Collect all price points
    price_list = [
        row['ben_soliman_price'], row['final_min_price'], row['final_mod_price'],
        row['final_max_price'], row['final_true_min'], row['final_true_max'],
        row['min_scrapped'], row['scrapped25'], row['scrapped50'], row['scrapped75'], row['max_scrapped']
    ]
    
    # Filter valid prices within acceptable range
    valid_prices = sorted({
        x for x in price_list 
        if x and not pd.isna(x) and x != 0 
        and wac / (1 - (avg_margin - (10 * std))) <= x <= wac / (1 - (max_marg + 10 * std))
        and x >= wac * 0.9
    })
    
    if not valid_prices:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    return (
        np.min(valid_prices),
        np.percentile(valid_prices, 25),
        np.percentile(valid_prices, 50),
        np.percentile(valid_prices, 75),
        np.max(valid_prices)
    )


def calculate_step_bounds(row):
    """Calculate below/above market bounds based on price steps."""
    wac = row['wac_p']
    std = row['std']
    prices = [row['minimum'], row['percentile_25'], row['percentile_50'], row['percentile_75'], row['maximum']]
    
    # Calculate valid steps between price points
    valid_steps = []
    for i in range(len(prices) - 1):
        step = prices[i + 1] - prices[i]
        if (step / wac) <= std * 1.2:
            valid_steps.append(step)
    
    avg_step = np.mean(valid_steps) if valid_steps else min(2 * std, 0.2 * row['target_margin'])
    
    new_min = prices[0] - avg_step if (prices[0] - avg_step) >= wac else prices[0]
    new_max = prices[-1] + avg_step if (prices[-1] + avg_step) >= wac else prices[-1]
    
    return new_min, new_max


In [20]:
# =============================================================================
# Apply Price Analysis & Margin Calculation (on market_data)
# =============================================================================

# Apply price analysis to calculate price percentiles
market_data[['minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum']] = \
    market_data.apply(price_analysis, axis=1, result_type='expand')

# Filter out records without valid price analysis
market_data = market_data[~market_data['minimum'].isna()]

# Calculate below/above market bounds
market_data[['below_market', 'above_market']] = market_data.apply(calculate_step_bounds, axis=1, result_type='expand')

print(f"Market data after price analysis: {len(market_data)} records")


Market data after price analysis: 12354 records


In [21]:
# =============================================================================
# Convert prices to margins (on market_data) - FINALIZE market_data processing
# =============================================================================

market_data['below_market'] = (market_data['below_market'] - market_data['wac_p']) / market_data['below_market']
market_data['market_min'] = (market_data['minimum'] - market_data['wac_p']) / market_data['minimum']
market_data['market_25'] = (market_data['percentile_25'] - market_data['wac_p']) / market_data['percentile_25']
market_data['market_50'] = (market_data['percentile_50'] - market_data['wac_p']) / market_data['percentile_50']
market_data['market_75'] = (market_data['percentile_75'] - market_data['wac_p']) / market_data['percentile_75']
market_data['market_max'] = (market_data['maximum'] - market_data['wac_p']) / market_data['maximum']
market_data['above_market'] = (market_data['above_market'] - market_data['wac_p']) / market_data['above_market']

# Select only the market-related columns to merge later
market_columns = [
    'cohort_id', 'product_id',
    # Market Prices (raw)
    'ben_soliman_price', 
    'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max',
    'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped',
    # Price Percentiles
    'minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum',
    # Margin Tiers
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market'
]
market_data = market_data[[c for c in market_columns if c in market_data.columns]]

print(f"\n{'='*60}")
print(f"MARKET DATA PROCESSING COMPLETE")
print(f"{'='*60}")
print(f"Total processed market records: {len(market_data)}")
print(f"\nMarket data columns:")
print("  - Price columns: ben_soliman_price, final_min_price, final_max_price, etc.")
print("  - Percentiles: minimum, percentile_25, percentile_50, percentile_75, maximum")
print("  - Margin tiers: below_market, market_min, market_25, market_50, market_75, market_max, above_market")
print(f"\nSample processed market data:")
market_data.head()



MARKET DATA PROCESSING COMPLETE
Total processed market records: 12354

Market data columns:
  - Price columns: ben_soliman_price, final_min_price, final_max_price, etc.
  - Percentiles: minimum, percentile_25, percentile_50, percentile_75, maximum
  - Margin tiers: below_market, market_min, market_25, market_50, market_75, market_max, above_market

Sample processed market data:


,cohort_id,product_id,ben_soliman_price,final_min_price,final_max_price,final_mod_price,final_true_min,final_true_max,min_scrapped,scrapped25,...,percentile_50,percentile_75,maximum,below_market,market_min,market_25,market_50,market_75,market_max,above_market
0,702,3.0,253.5,255.0,279.0,255.0,255.0,300.0,254.630005,255.285004,...,255.142502,255.776253,279.0,0.034136,0.037027,0.041649,0.043226,0.045597,0.125041,0.127414
1,702,9.0,NaN,831.4,848.4,829.0,829.0,850.0,NaN,NaN,...,839.900000,848.800000,850.0,0.005951,0.012246,0.014386,0.025065,0.035287,0.036649,0.042563
2,702,10.0,NaN,270.0,286.0,270.0,270.0,290.0,NaN,NaN,...,286.000000,288.000000,290.0,0.023419,0.030653,0.058548,0.084882,0.091237,0.097505,0.103686
4,702,14.0,465.0,461.5,477.0,477.0,460.0,477.0,NaN,NaN,...,463.250000,468.000000,477.0,0.022783,0.028448,0.030818,0.035264,0.045056,0.063074,0.068282
5,702,17.0,599.0,599.0,631.4,595.0,595.0,639.0,599.000000,599.000000,...,607.250000,626.050000,639.0,0.013675,0.022585,0.031336,0.042303,0.071062,0.089888,0.097479


## PART B: Build Main pricing_data DataFrame
Start with df_product_base (all our SKUs) and LEFT JOIN the processed market_data


In [22]:
# =============================================================================
# PART B: Build Main pricing_data DataFrame from df_product_base
# =============================================================================
print("Building main pricing_data DataFrame...")

# =============================================================================
# Step 1: Start with df_product_base as the MAIN dataframe (all our SKUs)
# =============================================================================
print("  Step 1: Starting with product base (all SKUs)...")
pricing_data = df_product_base.copy()
print(f"     Product base: {len(pricing_data)} records")

# =============================================================================
# Step 2: Add warehouse mapping (warehouse_id and warehouse name)
# =============================================================================
print("  Step 2: Adding warehouse mapping...")
warehouse_df = get_warehouse_df()
pricing_data = pricing_data.merge(
    warehouse_df[['cohort_id', 'warehouse_id', 'warehouse']], 
    on='cohort_id'
)
print(f"     After warehouse mapping: {len(pricing_data)} records")

# =============================================================================
# Step 3: LEFT JOIN processed market_data
# =============================================================================
print("  Step 3: Left joining processed market data...")
pricing_data = pricing_data.merge(
    market_data, 
    on=['cohort_id', 'product_id'], 
    how='left'
)
print(f"     After market data join: {len(pricing_data)} records")

# =============================================================================
# Step 4: LEFT JOIN supporting data (sales, margins, targets, groups)
# =============================================================================
print("  Step 4: Left joining supporting data...")

# Merge sales data (nmv only)
pricing_data = pricing_data.merge(
    df_sales[['cohort_id', 'product_id', 'nmv']], 
    on=['cohort_id', 'product_id'], 
    how='left'
)
pricing_data['nmv'] = pricing_data['nmv'].fillna(0)

# Merge margin statistics (by cohort_id + product_id)
pricing_data = pricing_data.merge(df_margin_stats, on=['cohort_id', 'product_id'], how='left')

# Merge target margins (by brand + cat)
pricing_data = pricing_data.merge(df_targets, on=['brand', 'cat'], how='left')
pricing_data['target_margin'] = pricing_data['target_bm'].fillna(pricing_data['cat_target_margin']).fillna(0)
pricing_data = pricing_data.drop(columns=['target_bm', 'cat_target_margin'], errors='ignore')

# Fill NaN values with defaults
pricing_data['std'] = pricing_data['std'].fillna(0.01)
pricing_data['avg_margin'] = pricing_data['avg_margin'].fillna(0)

# Merge product groups
pricing_data = pricing_data.merge(df_groups, on='product_id', how='left')

# =============================================================================
# Step 5: Calculate current margin
# =============================================================================
pricing_data['current_margin'] = (pricing_data['current_price'] - pricing_data['wac_p']) / pricing_data['current_price']

# Remove duplicates
pricing_data = pricing_data.drop_duplicates(subset=['cohort_id', 'product_id'])

# =============================================================================
# Reorder columns
# =============================================================================
final_columns = [
    # Product Base Info
    'cohort_id', 'product_id', 'region', 'warehouse_id', 'warehouse', 'sku', 'brand', 'cat',
    # Cost & Price
    'wac1', 'wac_p', 'current_price', 'current_margin',
    # Sales
    'nmv',
    # Market Prices (raw)
    'ben_soliman_price', 
    'final_min_price', 'final_max_price', 'final_mod_price', 'final_true_min', 'final_true_max',
    'min_scrapped', 'scrapped25', 'scrapped50', 'scrapped75', 'max_scrapped',
    # Price Percentiles
    'minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum',
    # Margin Tiers
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market',
    # Supporting Data
    'std', 'avg_margin', 'target_margin', 'group'
]
pricing_data = pricing_data[[c for c in final_columns if c in pricing_data.columns]]

print(f"\n{'='*60}")
print(f"PRICING DATA COMPLETE")
print(f"{'='*60}")
print(f"Total records: {len(pricing_data)}")
print(f"\nRecords with market data: {len(pricing_data[~pricing_data['minimum'].isna()])}")
print(f"Records without market data: {len(pricing_data[pricing_data['minimum'].isna()])}")
print(f"\nRecords with sales (nmv > 0): {len(pricing_data[pricing_data['nmv'] > 0])}")
print(f"Records without sales (nmv = 0): {len(pricing_data[pricing_data['nmv'] == 0])}")
print(f"\nSample data:")
pricing_data.head()


Building main pricing_data DataFrame...
  Step 1: Starting with product base (all SKUs)...
     Product base: 101994 records
  Step 2: Adding warehouse mapping...
     After warehouse mapping: 86122 records
  Step 3: Left joining processed market data...
     After market data join: 86122 records
  Step 4: Left joining supporting data...

PRICING DATA COMPLETE
Total records: 64589

Records with market data: 12354
Records without market data: 52235

Records with sales (nmv > 0): 20668
Records without sales (nmv = 0): 43921

Sample data:


,cohort_id,product_id,region,warehouse_id,warehouse,sku,brand,cat,wac1,wac_p,...,below_market,market_min,market_25,market_50,market_75,market_max,above_market,std,avg_margin,target_margin
0,703,11769,Delta West,337,El-Mahala,اوكسى يدوى لافندر - 330 جم,اوكسي,منظفات,178.345927,172.711963,...,-0.017243,-0.017243,0.017286,0.021739,0.044735,0.09099,0.094801,0.007115,0.046815,0.060000
2,703,12473,Delta West,337,El-Mahala,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,مولبد,الفوط الصحية,26.852206,24.478930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010000,0.048311,0.047400
4,1124,19964,Upper Egypt,501,Assiut FC,مولفيكس بريميوم حديث الولادة مقاس 1 - 58 حفاضة,مولفيكس,حفاضات أطفال,277.356428,253.363816,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007770,0.048400,0.043200
5,703,4966,Delta West,337,El-Mahala,اد-مي صلصة زجاج - 300 جم,اد-مي,صلصة و صوص,278.613980,272.201304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015532,0.037348,0.045000
7,704,18964,Delta East,339,Mansoura FC,كاتشب هاينز - 6 جرام,هاينز,صلصة و صوص,454.335604,423.577161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009910,0.036704,0.040782


## Discount Analysis - Price & Margin After Discount


In [23]:
# =============================================================================
# Discount Query - Get discount percentage by warehouse and product
# =============================================================================
DISCOUNT_QUERY = f'''
SELECT warehouse_id, product_id, total_discount/total_nmv AS discount_perc
FROM (
    SELECT  
        pso.warehouse_id,
        pso.product_id,
        SUM(pso.total_price) AS total_nmv,
        SUM((ITEM_QUANTITY_DISCOUNT_VALUE * pso.purchased_item_count) + 
            (ITEM_DISCOUNT_VALUE * pso.purchased_item_count)) AS total_discount
    FROM product_sales_order pso 
    JOIN sales_orders so ON so.id = pso.sales_order_id
    WHERE so.created_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1 
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY ALL
)
WHERE total_nmv > 0
'''

# Execute discount query
print("Loading discount data...")
df_discount = query_snowflake(DISCOUNT_QUERY)
df_discount = convert_to_numeric(df_discount)
print(f"Loaded {len(df_discount)} discount records")


Loading discount data...
Loaded 10463 discount records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [24]:
# =============================================================================
# Create pricing_with_discount DataFrame
# =============================================================================
print("Creating pricing_with_discount DataFrame...")

# Copy pricing_data
pricing_with_discount = pricing_data.copy()

# Merge discount data (by warehouse_id + product_id)
pricing_with_discount = pricing_with_discount.merge(
    df_discount[['warehouse_id', 'product_id', 'discount_perc']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing discount_perc with 0 (no discount)
pricing_with_discount['discount_perc'] = pricing_with_discount['discount_perc'].fillna(0)

# =============================================================================
# Calculate price and margin after discount
# =============================================================================
# Price after discount = current_price * (1 - discount_perc)
pricing_with_discount['price_after_discount'] = (
    pricing_with_discount['current_price'] * (1 - pricing_with_discount['discount_perc'])
)

# Margin after discount = (price_after_discount - wac_p) / price_after_discount
pricing_with_discount['margin_after_discount'] = (
    (pricing_with_discount['price_after_discount'] - pricing_with_discount['wac_p']) / 
    pricing_with_discount['price_after_discount']
)

print(f"\n{'='*60}")
print(f"PRICING WITH DISCOUNT DATA COMPLETE")
print(f"{'='*60}")
print(f"Total records: {len(pricing_with_discount)}")
print(f"Records with discount (discount_perc > 0): {len(pricing_with_discount[pricing_with_discount['discount_perc'] > 0])}")
print(f"Records without discount: {len(pricing_with_discount[pricing_with_discount['discount_perc'] == 0])}")
print(f"\nNew columns added:")
print("  - discount_perc: discount percentage from sales")
print("  - price_after_discount: current_price * (1 - discount_perc)")
print("  - margin_after_discount: (price_after_discount - wac_p) / price_after_discount")
print(f"\nSample data with discounts:")
pricing_with_discount[pricing_with_discount['discount_perc'] > 0][
    ['product_id', 'warehouse_id', 'current_price', 'current_margin', 
     'discount_perc', 'price_after_discount', 'margin_after_discount']
].head(10)


Creating pricing_with_discount DataFrame...

PRICING WITH DISCOUNT DATA COMPLETE
Total records: 64589
Records with discount (discount_perc > 0): 3019
Records without discount: 61570

New columns added:
  - discount_perc: discount percentage from sales
  - price_after_discount: current_price * (1 - discount_perc)
  - margin_after_discount: (price_after_discount - wac_p) / price_after_discount

Sample data with discounts:


,product_id,warehouse_id,current_price,current_margin,discount_perc,price_after_discount,margin_after_discount
5,23039,1,26.00,0.085596,0.010200,25.734800,0.076173
6,6494,236,577.00,0.042672,0.002678,575.454767,0.040101
13,6494,632,581.75,0.050489,0.005752,578.403744,0.044995
19,161,339,41.75,0.093907,0.011688,41.262039,0.083192
34,6872,797,270.25,0.060493,0.016900,265.682775,0.044342
35,23032,1,67.50,0.094259,0.037900,64.941748,0.058579
36,9381,401,33.50,0.047736,0.000434,33.485475,0.047323
43,11490,797,277.50,0.040033,0.010374,274.621128,0.029969
52,435,1,315.00,0.123261,0.015533,310.107000,0.109427
53,590,337,50.75,0.037626,0.007833,50.352473,0.030028


In [25]:
# =============================================================================
# Price Position - Determine where price_after_discount falls in market tiers
# =============================================================================

def get_price_position(row):
    """Determine the price position relative to market price tiers."""
    price = row['price_after_discount']
    wac = row['wac_p']
    
    # Check if we have market data (minimum price exists)
    if pd.isna(row['minimum']) or pd.isna(price):
        return "No Market Data"
    
    # Get price tiers
    min_price = row['minimum']
    p25 = row['percentile_25']
    p50 = row['percentile_50']
    p75 = row['percentile_75']
    max_price = row['maximum']
    
    # Calculate below_market and above_market prices from margins
    # margin = (price - wac) / price  =>  price = wac / (1 - margin)
    below_market_margin = row['below_market']
    above_market_margin = row['above_market']
    
    below_market_price = wac / (1 - below_market_margin) if below_market_margin < 1 else min_price
    above_market_price = wac / (1 - above_market_margin) if above_market_margin < 1 else max_price
    
    # Determine position based on price tiers
    if price < below_market_price:
        return "Below Market"
    elif price < min_price:
        return "Below Min"
    elif price < p25:
        return "At Min"
    elif price < p50:
        return "At 25th"
    elif price < p75:
        return "At 50th"
    elif price < max_price:
        return "At 75th"
    elif price < above_market_price:
        return "At Max"
    else:
        return "Above Market"

# Apply price position function
pricing_with_discount['price_position'] = pricing_with_discount.apply(get_price_position, axis=1)

# Summary of price positions
print(f"\n{'='*60}")
print(f"PRICE POSITION ANALYSIS")
print(f"{'='*60}")
print("\nPrice Position Distribution:")
print(pricing_with_discount['price_position'].value_counts().to_string())
print(f"\nPrice Position Percentages:")
print((pricing_with_discount['price_position'].value_counts(normalize=True) * 100).round(2).astype(str) + '%')

# Sample data showing price position
print(f"\nSample data with price position:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 'current_price', 'discount_perc', 
     'price_after_discount', 'minimum', 'maximum', 'price_position']
].head(15)



PRICE POSITION ANALYSIS

Price Position Distribution:
price_position
No Market Data    52235
At 75th            2293
At 50th            2235
At Max             2164
Above Market       2130
At 25th            1194
Below Market       1179
At Min             1012
Below Min           147

Price Position Percentages:
price_position
No Market Data    80.87%
At 75th            3.55%
At 50th            3.46%
At Max             3.35%
Above Market        3.3%
At 25th            1.85%
Below Market       1.83%
At Min             1.57%
Below Min          0.23%
Name: proportion, dtype: object

Sample data with price position:


,product_id,warehouse_id,sku,current_price,discount_perc,price_after_discount,minimum,maximum,price_position
0,11769,337,اوكسى يدوى لافندر - 330 جم,186.75,0.000000,186.750000,169.784363,190.0,At 75th
1,12473,337,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,25.75,0.000000,25.750000,NaN,NaN,No Market Data
2,19964,501,مولفيكس بريميوم حديث الولادة مقاس 1 - 58 حفاضة,271.75,0.000000,271.750000,NaN,NaN,No Market Data
3,4966,337,اد-مي صلصة زجاج - 300 جم,285.00,0.000000,285.000000,NaN,NaN,No Market Data
4,18964,339,كاتشب هاينز - 6 جرام,436.00,0.000000,436.000000,NaN,NaN,No Market Data
5,23039,1,البوادي مربى كريمي تين - 365 جرام,26.00,0.010200,25.734800,NaN,NaN,No Market Data
6,6494,236,سندة زيت خليط - 700 مل,577.00,0.002678,575.454767,NaN,NaN,No Market Data
7,20486,337,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,50.00,0.000000,50.000000,47.160000,50.0,At Max
8,24122,632,لبان ترايدنت ليمون و توت 7 قطع - طعم جديد 7 قطعه,183.75,0.000000,183.750000,NaN,NaN,No Market Data
9,142,339,عصير جهينة كوكتيل - 235 مل,193.50,0.000000,193.500000,190.000000,205.0,At 25th


In [26]:
# =============================================================================
# Stock Query - Get available stock by warehouse and product
# =============================================================================
STOCK_QUERY = '''
SELECT 
    pw.warehouse_id,
    pw.product_id,
    pw.available_stock::INTEGER AS stocks
FROM product_warehouse pw
WHERE pw.warehouse_id NOT IN (6, 9, 10)
    AND pw.is_basic_unit = 1
'''

# Execute stock query
print("Loading stock data...")
df_stocks = query_snowflake(STOCK_QUERY)
df_stocks = convert_to_numeric(df_stocks)
print(f"Loaded {len(df_stocks)} stock records")

# Merge stock data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_stocks[['warehouse_id', 'product_id', 'stocks']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing stocks with 0
pricing_with_discount['stocks'] = pricing_with_discount['stocks'].fillna(0).astype(int)

print(f"\nStock data merged!")
print(f"Records with stock (stocks > 0): {len(pricing_with_discount[pricing_with_discount['stocks'] > 0])}")
print(f"Records without stock (stocks = 0): {len(pricing_with_discount[pricing_with_discount['stocks'] == 0])}")
print(f"\nSample data with stocks:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 'stocks', 'price_after_discount', 'price_position']
].head(10)


Loading stock data...


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Loaded 1837903 stock records

Stock data merged!
Records with stock (stocks > 0): 14100
Records without stock (stocks = 0): 50489

Sample data with stocks:


,product_id,warehouse_id,sku,stocks,price_after_discount,price_position
0,11769,337,اوكسى يدوى لافندر - 330 جم,0,186.750000,At 75th
1,12473,337,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,0,25.750000,No Market Data
2,19964,501,مولفيكس بريميوم حديث الولادة مقاس 1 - 58 حفاضة,39,271.750000,No Market Data
3,4966,337,اد-مي صلصة زجاج - 300 جم,10,285.000000,No Market Data
4,18964,339,كاتشب هاينز - 6 جرام,29,436.000000,No Market Data
5,23039,1,البوادي مربى كريمي تين - 365 جرام,133,25.734800,No Market Data
6,6494,236,سندة زيت خليط - 700 مل,77,575.454767,No Market Data
7,20486,337,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,76,50.000000,At Max
8,24122,632,لبان ترايدنت ليمون و توت 7 قطع - طعم جديد 7 قطعه,0,183.750000,No Market Data
9,142,339,عصير جهينة كوكتيل - 235 مل,67,193.500000,At 25th


In [27]:
# =============================================================================
# Zero Demand Query - Identify SKUs with zero/low demand
# =============================================================================
ZERO_DEMAND_QUERY = f'''
WITH last_oss AS (
    SELECT product_id, warehouse_id, TIMESTAMP AS last_in_stock_day
    FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY product_id, warehouse_id ORDER BY TIMESTAMP DESC) AS rnk 
        FROM materialized_views.STOCK_DAY_CLOSE
        WHERE AVAILABLE_STOCK = 0 
            AND TIMESTAMP >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120
        QUALIFY rnk = 1 
    )
),

current_stocks AS (
    SELECT product_id, warehouse_id, AVAILABLE_STOCK, activation
    FROM PRODUCT_WAREHOUSE
    WHERE IS_BASIC_UNIT = 1
        AND CASE WHEN product_id = 1309 THEN packing_unit_id <> 23 ELSE TRUE END
),

prs AS (
    SELECT DISTINCT 
        product_purchased_receipts.product_id,
        purchased_receipts.warehouse_id,
        purchased_receipts.date::DATE AS date,
        product_purchased_receipts.purchased_item_count * product_purchased_receipts.basic_unit_count AS purchase_min_count
    FROM product_purchased_receipts
    JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
    JOIN last_oss lo ON product_purchased_receipts.product_id = lo.product_id 
        AND lo.warehouse_id = purchased_receipts.warehouse_id 
        AND purchased_receipts.date > lo.last_in_stock_day 
    WHERE product_purchased_receipts.purchased_item_count <> 0
        AND purchased_receipts.purchased_receipt_status_id IN (4, 5, 7)
        AND purchased_receipts.date::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120
),

main AS (
    SELECT 
        prs.product_id, 
        prs.warehouse_id, 
        MIN(date) AS first_order_date, 
        SUM(purchase_min_count) AS total_recieved, 
        cs.AVAILABLE_STOCK, 
        cs.activation
    FROM prs 
    JOIN current_stocks cs ON cs.product_id = prs.product_id AND prs.warehouse_id = cs.warehouse_id
    GROUP BY prs.product_id, prs.warehouse_id, cs.AVAILABLE_STOCK, cs.activation
),

sold_days AS (
    SELECT product_id, warehouse_id, COUNT(DISTINCT o_date) AS sales_days
    FROM (
        SELECT DISTINCT
            so.created_at::DATE AS o_date,
            pso.warehouse_id,
            pso.product_id,
            SUM(pso.purchased_item_count * basic_unit_count) AS daily_qty
        FROM product_sales_order pso
        JOIN sales_orders so ON so.id = pso.sales_order_id
        JOIN main m ON m.product_id = pso.product_id 
            AND m.warehouse_id = pso.warehouse_id 
            AND so.created_at::DATE >= m.first_order_date
        WHERE so.created_at::DATE BETWEEN CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120 
            AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
            AND so.sales_order_status_id NOT IN (7, 12)
            AND so.channel IN ('telesales', 'retailer')
            AND pso.purchased_item_count <> 0
        GROUP BY o_date, pso.warehouse_id, pso.product_id
    )
    GROUP BY product_id, warehouse_id
)

SELECT DISTINCT warehouse_id, product_id
FROM (
    SELECT m.product_id, m.warehouse_id, m.first_order_date, m.activation,
        COALESCE(sd.sales_days, 0) AS sales_days,
        COALESCE(sd.sales_days, 0)::FLOAT / NULLIF((CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1) - m.first_order_date, 0) AS perc_days
    FROM main m 
    LEFT JOIN sold_days sd ON sd.product_id = m.product_id AND sd.warehouse_id = m.warehouse_id
    WHERE m.first_order_date < CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 10
)
WHERE perc_days <= 0.3
    AND activation = 'true'
'''

# Execute zero demand query
print("Loading zero demand SKUs...")
df_zero_demand = query_snowflake(ZERO_DEMAND_QUERY)
df_zero_demand = convert_to_numeric(df_zero_demand)
print(f"Loaded {len(df_zero_demand)} zero demand SKU records")


Loading zero demand SKUs...
Loaded 3915 zero demand SKU records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [28]:
# =============================================================================
# Add Zero Demand Flag to pricing_with_discount
# =============================================================================

# Add a marker column to identify zero demand SKUs
df_zero_demand['zero_demand'] = 1

# Merge with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_zero_demand[['warehouse_id', 'product_id', 'zero_demand']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing values with 0 (not zero demand)
pricing_with_discount['zero_demand'] = pricing_with_discount['zero_demand'].fillna(0).astype(int)

print(f"Zero demand flag added!")
print(f"SKUs flagged as zero demand: {len(pricing_with_discount[pricing_with_discount['zero_demand'] == 1])}")
print(f"SKUs with normal demand: {len(pricing_with_discount[pricing_with_discount['zero_demand'] == 0])}")


Zero demand flag added!
SKUs flagged as zero demand: 2766
SKUs with normal demand: 61823


In [29]:
# =============================================================================
# OOS Yesterday Query - Identify SKUs out of stock yesterday
# =============================================================================
OOS_YESTERDAY_QUERY = f'''
SELECT DISTINCT product_id, warehouse_id,
    CASE WHEN opening_stocks = 0 AND closing_stocks = 0 THEN 1
         ELSE 0 
    END AS oos_yesterday
FROM (
    SELECT 
        timestamp,
        product_id,
        warehouse_id, 
        AVAILABLE_STOCK AS closing_stocks,
        LAG(AVAILABLE_STOCK) OVER (PARTITION BY product_id, warehouse_id ORDER BY TIMESTAMP) AS opening_stocks
    FROM materialized_views.stock_day_close
    WHERE timestamp::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 2
    QUALIFY opening_stocks IS NOT NULL
)
WHERE oos_yesterday = 1
'''

# Execute OOS yesterday query
print("Loading OOS yesterday data...")
df_oos_yesterday = query_snowflake(OOS_YESTERDAY_QUERY)
df_oos_yesterday = convert_to_numeric(df_oos_yesterday)
print(f"Loaded {len(df_oos_yesterday)} OOS yesterday records")


Loading OOS yesterday data...
Loaded 1886200 OOS yesterday records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [30]:
# =============================================================================
# Add OOS Yesterday Flag to pricing_with_discount
# =============================================================================

# Merge with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_oos_yesterday[['warehouse_id', 'product_id', 'oos_yesterday']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing values with 0 (not OOS yesterday)
pricing_with_discount['oos_yesterday'] = pricing_with_discount['oos_yesterday'].fillna(0).astype(int)

print(f"OOS yesterday flag added!")
print(f"SKUs out of stock yesterday: {len(pricing_with_discount[pricing_with_discount['oos_yesterday'] == 1])}")
print(f"SKUs in stock yesterday: {len(pricing_with_discount[pricing_with_discount['oos_yesterday'] == 0])}")


OOS yesterday flag added!
SKUs out of stock yesterday: 50359
SKUs in stock yesterday: 14230


In [31]:
# =============================================================================
# Running Rate Query - Get in-stock running rate by warehouse and product
# =============================================================================
RUNNING_RATE_QUERY = f'''
WITH params AS (
    SELECT
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS run_date,
        DATEADD(month, -3, CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE) AS history_start
),

-- Daily sales aggregation
sales_base AS (
    SELECT
        pso.product_id,
        pso.warehouse_id,
        DATE_TRUNC('day', pso.created_at)::DATE AS date,
        SUM(pso.purchased_item_count * pso.basic_unit_count) AS sold_units,
        SUM(pso.purchased_item_count * pso.basic_unit_count * pso.item_price)
            / NULLIF(SUM(pso.purchased_item_count * pso.basic_unit_count), 0) AS avg_selling_price,
        COUNT(DISTINCT so.retailer_id) AS retailer_count
    FROM product_sales_order pso
    JOIN sales_orders so ON pso.sales_order_id = so.id
    WHERE DATE_TRUNC('day', pso.created_at)::DATE >= (SELECT history_start FROM params)
    GROUP BY 1, 2, 3
),

-- Stock daily metrics
stock_daily AS (
    SELECT
        product_id,
        warehouse_id,
        DATE_TRUNC('day', TIMESTAMP)::DATE AS date,
        MAX_BY(available_stock, TIMESTAMP) AS stock_closing,
        24 * SUM(CASE WHEN activation = FALSE OR available_stock = 0 THEN 1 ELSE 0 END)::FLOAT 
            / NULLIF(COUNT(*), 0) AS oos_hours,
        MAX(CASE WHEN activation = TRUE AND available_stock > 0 THEN 1 ELSE 0 END) AS in_stock_flag
    FROM materialized_views.STOCK_SNAP_SHOTS_RECENT
    WHERE product_id IS NOT NULL
    GROUP BY product_id, warehouse_id, date
),

-- Join sales + stock + WAC (only in-stock days)
base_data AS (
    SELECT
        sb.product_id,
        sb.warehouse_id,
        sb.date,
        sb.sold_units,
        sb.avg_selling_price,
        sb.retailer_count,
        sd.oos_hours,
        sd.in_stock_flag,
        ac.wac_p AS wac,
        CASE WHEN DAYOFWEEKISO(sb.date) IN (5, 6) THEN 1 ELSE 0 END AS is_weekend
    FROM sales_base sb
    LEFT JOIN stock_daily sd ON sb.product_id = sd.product_id 
        AND sb.warehouse_id = sd.warehouse_id AND sb.date = sd.date
    LEFT JOIN finance.ALL_COGS ac ON sb.product_id = ac.product_id 
        AND sb.date BETWEEN ac.from_date AND ac.to_date
    WHERE sd.in_stock_flag = 1
),

-- Stats per SKU x Warehouse
sku_wh_stats AS (
    SELECT
        product_id, warehouse_id,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY sold_units) AS med_units,
        PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY sold_units) AS pct95_units,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY retailer_count) AS med_retailers,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY 
            CASE WHEN avg_selling_price IS NULL OR avg_selling_price = 0 THEN 0 
            ELSE (avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0) END
        ) AS med_margin
    FROM base_data
    GROUP BY product_id, warehouse_id
),

-- Cap outliers and adjust for retailer spikes
adjusted AS (
    SELECT
        b.product_id, b.warehouse_id, b.date, b.in_stock_flag, b.oos_hours, b.is_weekend,
        b.avg_selling_price, b.wac, s.med_margin,
        CASE 
            WHEN b.retailer_count > GREATEST(2, s.med_retailers * 2) 
                AND b.retailer_count > 0 AND s.med_retailers IS NOT NULL
            THEN ROUND(LEAST(b.sold_units, s.pct95_units) * (s.med_retailers::FLOAT / NULLIF(b.retailer_count::FLOAT, 0)), 0)
            ELSE LEAST(b.sold_units, s.pct95_units)
        END AS units_adjusted
    FROM base_data b
    LEFT JOIN sku_wh_stats s ON b.product_id = s.product_id AND b.warehouse_id = s.warehouse_id
),

-- Apply weights (recency, stock availability, weekend, margin)
weighted AS (
    SELECT
        product_id, warehouse_id, date, units_adjusted,
        (
            -- Recency weight
            CASE WHEN date >= DATEADD(day, -21, (SELECT run_date FROM params)) THEN 1.5
                 WHEN date >= DATEADD(day, -90, (SELECT run_date FROM params)) THEN 1.0
                 ELSE 0.5 END
            -- In-stock weight
            * CASE WHEN in_stock_flag = 1 AND COALESCE(oos_hours, 0) < 12 THEN 1.4
                   WHEN in_stock_flag = 1 AND COALESCE(oos_hours, 0) >= 12 THEN 0.9
                   ELSE 0.6 END
            -- Weekend weight
            * CASE WHEN is_weekend = 1 THEN 0.7 ELSE 1.0 END
            -- Margin weight
            * CASE WHEN avg_selling_price IS NULL OR avg_selling_price = 0 OR med_margin IS NULL THEN 1.0
                   WHEN ((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0)) < med_margin
                   THEN 1.0 + LEAST((med_margin - ((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0))) * 2.0, 0.6)
                   WHEN ((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0)) > med_margin
                   THEN 1.0 - LEAST((((avg_selling_price - COALESCE(wac, 0)) / NULLIF(avg_selling_price, 0)) - med_margin) * 2.0, 0.4)
                   ELSE 1.0 END
        ) AS final_weight
    FROM adjusted
    WHERE units_adjusted IS NOT NULL
),

-- Weighted average forecast
forecast_base AS (
    SELECT
        product_id, warehouse_id,
        SUM(units_adjusted * final_weight) / NULLIF(SUM(final_weight), 0) AS weighted_avg_units
    FROM weighted
    GROUP BY product_id, warehouse_id
),

-- Zero-sales last 4 days (with stock) exclusion flag
last4_flag AS (
    SELECT product_id, warehouse_id,
        CASE WHEN COUNT(*) = 4 
             AND SUM(CASE WHEN COALESCE(sold_units, 0) = 0 AND in_stock_flag = 1 THEN 1 ELSE 0 END) = 4
        THEN 1 ELSE 0 END AS exclude_flag
    FROM base_data
    WHERE date >= DATEADD(day, -4, (SELECT run_date FROM params)) 
        AND date < (SELECT run_date FROM params)
    GROUP BY product_id, warehouse_id
),

-- Zero sales excluded (in stock but no sales)
zero_sales_excluded AS (
    SELECT DISTINCT s.warehouse_id, s.product_id
    FROM (
        SELECT pw.warehouse_id, pw.product_id, SUM(pw.available_stock)::INT AS stocks
        FROM product_warehouse pw
        WHERE pw.warehouse_id NOT IN (6, 9, 10) AND pw.is_basic_unit = 1 AND pw.available_stock > 0
        GROUP BY pw.warehouse_id, pw.product_id
    ) s
    LEFT JOIN (
        SELECT pso.product_id, pso.warehouse_id, SUM(pso.total_price) AS nmv
        FROM product_sales_order pso
        JOIN sales_orders so ON so.id = pso.sales_order_id
        WHERE so.created_at::date BETWEEN CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 5 
            AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1
            AND so.sales_order_status_id NOT IN (7, 12) AND so.channel IN ('telesales', 'retailer')
            AND pso.purchased_item_count <> 0
        GROUP BY pso.product_id, pso.warehouse_id
    ) md ON md.product_id = s.product_id AND md.warehouse_id = s.warehouse_id
    LEFT JOIN finance.all_cogs f ON f.product_id = s.product_id
        AND f.from_date::date <= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
        AND f.to_date::date > CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
    LEFT JOIN (
        SELECT pr.warehouse_id, ppr.product_id, SUM(ppr.final_price) AS total_prs
        FROM product_purchased_receipts ppr
        JOIN purchased_receipts pr ON pr.id = ppr.purchased_receipt_id
        WHERE pr.date::date >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 4
            AND pr.is_actual = 'true' AND pr.purchased_receipt_status_id IN (4, 5, 7)
            AND ppr.purchased_item_count <> 0
        GROUP BY pr.warehouse_id, ppr.product_id
    ) prs ON prs.product_id = s.product_id AND prs.warehouse_id = s.warehouse_id
    WHERE COALESCE(md.nmv, 0) = 0 
        AND COALESCE(prs.total_prs, 0) < 0.7 * (COALESCE(f.wac_p, 0) * s.stocks)
),

-- First sale date for new products
first_sale AS (
    SELECT product_id, warehouse_id, MIN(date) AS first_sale_date
    FROM base_data WHERE sold_units > 0
    GROUP BY product_id, warehouse_id
)

-- Final output: running rate per warehouse/product
SELECT
    fb.warehouse_id,
    fb.product_id,
    CASE
        WHEN l4.exclude_flag = 1 THEN 0
        WHEN fs.first_sale_date >= DATEADD(day, -2, (SELECT run_date FROM params))
        THEN GREATEST(CEIL(fb.weighted_avg_units), 1)
        ELSE CEIL(fb.weighted_avg_units)
    END AS In_stock_rr
FROM forecast_base fb
LEFT JOIN last4_flag l4 ON fb.product_id = l4.product_id AND fb.warehouse_id = l4.warehouse_id
LEFT JOIN first_sale fs ON fb.product_id = fs.product_id AND fb.warehouse_id = fs.warehouse_id
LEFT JOIN zero_sales_excluded zse ON fb.product_id = zse.product_id AND fb.warehouse_id = zse.warehouse_id
WHERE zse.product_id IS NULL
'''

# Execute running rate query
print("Loading running rate data (this may take a moment)...")
df_running_rate = query_snowflake(RUNNING_RATE_QUERY)
df_running_rate = convert_to_numeric(df_running_rate)
print(f"Loaded {len(df_running_rate)} running rate records")


Loading running rate data (this may take a moment)...
Loaded 22486 running rate records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [32]:
# =============================================================================
# Merge Running Rate and Calculate DOH (Days on Hand)
# =============================================================================

# Merge running rate data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_running_rate[['warehouse_id', 'product_id', 'in_stock_rr']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing running rate with 0
pricing_with_discount['in_stock_rr'] = pricing_with_discount['in_stock_rr'].fillna(0)

# Calculate DOH (Days on Hand) = stocks / in_stock_rr
# Handle division by zero - if running rate is 0, DOH is infinite (use 999)
pricing_with_discount['doh'] = np.select(
    [
        (pricing_with_discount['in_stock_rr'] > 0) & (pricing_with_discount['stocks'] > 0),
        pricing_with_discount['stocks'] == 0
    ],
    [
        pricing_with_discount['stocks'] / pricing_with_discount['in_stock_rr'],
        0
    ],
    default=999
)


In [33]:
# =============================================================================
# Product Classification Query - ABC Classification based on order contribution
# =============================================================================
PRODUCT_CLASSIFICATION_QUERY = f'''
WITH order_counts AS (
    SELECT 
        pso.warehouse_id,
        pso.product_id,
        COUNT(DISTINCT pso.sales_order_id) AS order_count
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    WHERE so.created_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 90
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY pso.warehouse_id, pso.product_id
),

warehouse_totals AS (
    SELECT 
        warehouse_id,
        SUM(order_count) AS total_orders
    FROM order_counts
    GROUP BY warehouse_id
),

ranked_products AS (
    SELECT 
        oc.warehouse_id,
        oc.product_id,
        oc.order_count,
        wt.total_orders,
        oc.order_count::FLOAT / NULLIF(wt.total_orders, 0) AS contribution,
        SUM(oc.order_count::FLOAT / NULLIF(wt.total_orders, 0)) 
            OVER (PARTITION BY oc.warehouse_id ORDER BY oc.order_count DESC 
                  ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_contribution
    FROM order_counts oc
    JOIN warehouse_totals wt ON oc.warehouse_id = wt.warehouse_id
)

SELECT 
    warehouse_id,
    product_id,
    order_count,
    contribution,
    cumulative_contribution,
    CASE 
        WHEN cumulative_contribution <= 0.4 THEN 'A'
        WHEN cumulative_contribution <= 0.8 THEN 'B'
        ELSE 'C'
    END AS abc_class
FROM ranked_products
'''

# Execute product classification query
print("Loading product classification data...")
df_classification = query_snowflake(PRODUCT_CLASSIFICATION_QUERY)
df_classification = convert_to_numeric(df_classification)
print(f"Loaded {len(df_classification)} product classification records")
print(f"\nClassification distribution:")
print(df_classification['abc_class'].value_counts().to_string())


Loading product classification data...
Loaded 27477 product classification records

Classification distribution:
abc_class
C    20840
B     5480
A     1157


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [34]:
# =============================================================================
# Add ABC Classification to pricing_with_discount
# =============================================================================

# Merge classification data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_classification[['warehouse_id', 'product_id', 'order_count', 'contribution', 'abc_class']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing values - products without orders in last 3 months get class 'C'
pricing_with_discount['order_count'] = pricing_with_discount['order_count'].fillna(0).astype(int)
pricing_with_discount['contribution'] = pricing_with_discount['contribution'].fillna(0)
pricing_with_discount['abc_class'] = pricing_with_discount['abc_class'].fillna('C')

print(f"ABC Classification added!")
print(f"\nClassification in pricing_with_discount:")
print(pricing_with_discount['abc_class'].value_counts().to_string())
print(f"\nSample data with classification:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 'order_count', 'contribution', 'abc_class', 'stocks', 'doh']
].head(15)


ABC Classification added!

Classification in pricing_with_discount:
abc_class
C    60124
B     3693
A      772

Sample data with classification:


,product_id,warehouse_id,sku,order_count,contribution,abc_class,stocks,doh
0,11769,337,اوكسى يدوى لافندر - 330 جم,241,0.001105,B,0,0.000000
1,12473,337,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,0,0.000000,C,0,0.000000
2,19964,501,مولفيكس بريميوم حديث الولادة مقاس 1 - 58 حفاضة,46,0.000355,C,39,13.000000
3,4966,337,اد-مي صلصة زجاج - 300 جم,27,0.000124,C,10,5.000000
4,18964,339,كاتشب هاينز - 6 جرام,131,0.000497,B,29,9.666667
5,23039,1,البوادي مربى كريمي تين - 365 جرام,209,0.000231,C,133,10.230769
6,6494,236,سندة زيت خليط - 700 مل,356,0.000649,B,77,12.833333
7,20486,337,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,230,0.001054,B,76,8.444444
8,24122,632,لبان ترايدنت ليمون و توت 7 قطع - طعم جديد 7 قطعه,0,0.000000,C,0,0.000000
9,142,339,عصير جهينة كوكتيل - 235 مل,629,0.002385,A,67,4.785714


In [35]:
# =============================================================================
# PO (Purchase Order) Data Query - Last PO status and rejection count
# =============================================================================
PO_DATA_QUERY = '''
WITH last_data AS (
    SELECT product_id, warehouse_id, confirmation_status, PO_date::DATE AS last_po_date, ordered_qty
    FROM (
        SELECT 
            product_id,
            Target_WAREHOUSE_ID AS warehouse_id,
            confirmation_status,
            created_at AS PO_date,
            MIN_QUANTITY AS ordered_qty,
            reason,
            MAX(created_at) OVER (PARTITION BY product_id, Target_WAREHOUSE_ID) AS last_po
        FROM retool.PO_INITIAL_PLAN
        WHERE created_at::DATE >= CURRENT_DATE - 15 
    ) x
    WHERE last_po = PO_date
),

last_15_data AS (
    SELECT 
        product_id,
        target_WAREHOUSE_ID AS warehouse_id,
        COUNT(DISTINCT CASE WHEN confirmation_status <> 'yes' THEN created_at END) AS no_last_15
    FROM retool.PO_INITIAL_PLAN
    WHERE created_at::DATE >= CURRENT_DATE - 15 
    GROUP BY 1, 2
)

SELECT 
    ld.product_id,
    ld.warehouse_id,
    ld.confirmation_status,
    ld.last_po_date,
    ld.ordered_qty,
    COALESCE(lfd.no_last_15, 0) AS no_last_15
FROM last_data ld 
LEFT JOIN last_15_data lfd 
    ON lfd.product_id = ld.product_id 
    AND lfd.warehouse_id = ld.warehouse_id
'''

# Execute PO data query using dwh_pg_query
print("Loading PO data...")
df_po_data = setup_environment_2.dwh_pg_query(
    PO_DATA_QUERY, 
    columns=['product_id', 'warehouse_id', 'confirmation_status', 'last_po_date', 'ordered_qty', 'no_last_15']
)
df_po_data.columns = df_po_data.columns.str.lower()
df_po_data = convert_to_numeric(df_po_data)
print(f"Loaded {len(df_po_data)} PO records")
print(f"\nConfirmation status distribution:")
print(df_po_data['confirmation_status'].value_counts().to_string())


Loading PO data...
Loaded 17319 PO records

Confirmation status distribution:
confirmation_status
yes    13190
no      3809


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [36]:
# =============================================================================
# Add PO Data to pricing_with_discount
# =============================================================================

# Merge PO data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_po_data[['warehouse_id', 'product_id', 'confirmation_status', 'last_po_date', 'ordered_qty', 'no_last_15']], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill missing values
pricing_with_discount['ordered_qty'] = pricing_with_discount['ordered_qty'].fillna(0)
pricing_with_discount['no_last_15'] = pricing_with_discount['no_last_15'].fillna(0).astype(int)

print(f"PO data added!")
print(f"\nRecords with PO data: {len(pricing_with_discount[~pricing_with_discount['confirmation_status'].isna()])}")
print(f"Records without PO data: {len(pricing_with_discount[pricing_with_discount['confirmation_status'].isna()])}")
print(f"\nSample data with PO info:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 'confirmation_status', 'last_po_date', 'ordered_qty', 'no_last_15']
].dropna(subset=['confirmation_status']).head(15)


PO data added!

Records with PO data: 11437
Records without PO data: 53152

Sample data with PO info:


,product_id,warehouse_id,sku,confirmation_status,last_po_date,ordered_qty,no_last_15
3,4966,337,اد-مي صلصة زجاج - 300 جم,yes,2026-01-15,2.0,0
4,18964,339,كاتشب هاينز - 6 جرام,yes,2026-01-13,3.0,5
5,23039,1,البوادي مربى كريمي تين - 365 جرام,yes,2026-01-15,12.0,0
7,20486,337,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,yes,2026-01-15,42.0,1
9,142,339,عصير جهينة كوكتيل - 235 مل,yes,2026-01-14,20.0,2
13,6494,632,سندة زيت خليط - 700 مل,yes,2026-01-04,48.0,0
17,161,632,مرقة ماجى دجاج شريط - 8 جم,yes,2026-01-13,8658.0,0
18,62,401,نسكافيه كلاسيك - 1.8 جم,no,2026-01-15,48.0,1
19,161,339,مرقة ماجى دجاج شريط - 8 جم,yes,2026-01-13,54.0,0
25,28,1,مكرونة الملكة فرن - 1 كجم,yes,2026-01-05,26.0,0


In [37]:
# =============================================================================
# Leadtime Query - Supplier leadtime by brand, category, and warehouse
# =============================================================================
LEADTIME_QUERY = '''
SELECT brand, cat, warehouse_id, leadtime
FROM (
    SELECT a.*, b.name_ar AS brand, c.name_ar AS cat
    FROM (
        SELECT DISTINCT 
            sl.supplier_id, 
            warehouse_id, 
            category_id, 
            brand_id, 
            sl.updated_at, 
            leadtime,
            MAX(sl.updated_at) OVER (PARTITION BY sl.supplier_id, warehouse_id) AS last_update
        FROM retool.SUPPLIER_MOQ sl 
        JOIN retool.PO_SUPPLIER_MAPPING sm ON sl.supplier_id = sm.supplier_id 
    ) a
    JOIN brands b ON b.id = a.brand_id 
    JOIN categories c ON c.id = a.category_id
    WHERE a.updated_at = last_update
) d
'''

# Execute leadtime query using dwh_pg_query
print("Loading leadtime data...")
df_leadtime = setup_environment_2.dwh_pg_query(
    LEADTIME_QUERY, 
    columns=['brand', 'cat', 'warehouse_id', 'leadtime']
)
df_leadtime.columns = df_leadtime.columns.str.lower()
df_leadtime = convert_to_numeric(df_leadtime)
print(f"Loaded {len(df_leadtime)} leadtime records")


Loading leadtime data...
Loaded 14817 leadtime records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [ ]:
# =============================================================================
# Add Leadtime to pricing_with_discount
# =============================================================================

# Merge leadtime data with pricing_with_discount (by brand, cat, warehouse_id)
pricing_with_discount = pricing_with_discount.merge(
    df_leadtime[['brand', 'cat', 'warehouse_id', 'leadtime']], 
    on=['brand', 'cat', 'warehouse_id'], 
    how='left'
)

# Fill missing leadtime with 0 or a default value
pricing_with_discount['leadtime'] = pricing_with_discount['leadtime'].fillna(72)


print(f"Leadtime data added!")
print(f"\nRecords with leadtime: {len(pricing_with_discount[pricing_with_discount['leadtime'] > 0])}")
print(f"Records without leadtime: {len(pricing_with_discount[pricing_with_discount['leadtime'] == 0])}")
print(f"\nLeadtime distribution:")
print(pricing_with_discount['leadtime'].describe())

# =============================================================================
# Calculate Expected Receiving Day
# If confirmation_status is 'no': add 2 extra days (48 hours) before adding leadtime
# expected_receiving_day = last_po_date + ((2 + leadtime) / 24) if not confirmed
# expected_receiving_day = last_po_date + (leadtime / 24) if confirmed
# =============================================================================

# Convert last_po_date to datetime if not already
pricing_with_discount['last_po_date'] = pd.to_datetime(pricing_with_discount['last_po_date'], errors='coerce')

# Calculate adjusted leadtime: add 48 hours (2 days) if confirmation_status is 'no'
pricing_with_discount['adjusted_leadtime'] = np.where(
    pricing_with_discount['confirmation_status'].str.lower() == 'no',
    pricing_with_discount['leadtime'] + 48,  # Add 2 days (48 hours) if not confirmed
    pricing_with_discount['leadtime']
)

# Calculate expected receiving day (leadtime is in hours, divide by 24 for days)
pricing_with_discount['expected_receiving_day'] = pricing_with_discount['last_po_date'] + pd.to_timedelta(
    pricing_with_discount['adjusted_leadtime'] / 24, unit='D'
)

print(f"\nExpected receiving day calculated!")
print(f"Records with expected receiving day: {len(pricing_with_discount[~pricing_with_discount['expected_receiving_day'].isna()])}")
print(f"Records with confirmation_status='no' (added 2 extra days): {len(pricing_with_discount[pricing_with_discount['confirmation_status'].str.lower() == 'no'])}")
print(f"\nSample data with expected receiving day:")
pricing_with_discount[~pricing_with_discount['last_po_date'].isna()][
    ['product_id', 'warehouse_id', 'sku', 'confirmation_status', 'last_po_date', 'leadtime', 'adjusted_leadtime', 'expected_receiving_day', 'doh']
].head(15)


Leadtime data added!

Records with leadtime: 68380
Records without leadtime: 0

Leadtime distribution:
count    68380.000000
mean        55.402164
std         30.684771
min         24.000000
25%         48.000000
50%         48.000000
75%         72.000000
max        168.000000
Name: leadtime, dtype: float64

Expected receiving day calculated!
Records with expected receiving day: 12173

Sample data with expected receiving day:


,product_id,warehouse_id,sku,last_po_date,leadtime,expected_receiving_day,doh
0,11769,337,اوكسى يدوى لافندر - 330 جم,2026-01-06,24.0,2026-01-07,0.000000
3,4966,337,اد-مي صلصة زجاج - 300 جم,2026-01-15,48.0,2026-01-17,5.000000
4,18964,339,كاتشب هاينز - 6 جرام,2026-01-13,48.0,2026-01-15,9.666667
5,23039,1,البوادي مربى كريمي تين - 365 جرام,2026-01-15,48.0,2026-01-17,10.230769
7,20486,337,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,2026-01-15,24.0,2026-01-16,8.444444
9,142,339,عصير جهينة كوكتيل - 235 مل,2026-01-14,24.0,2026-01-15,4.785714
13,6494,632,سندة زيت خليط - 700 مل,2026-01-04,48.0,2026-01-06,11.000000
17,161,632,مرقة ماجى دجاج شريط - 8 جم,2026-01-13,48.0,2026-01-15,5.211692
18,62,401,نسكافيه كلاسيك - 1.8 جم,2026-01-15,48.0,2026-01-17,0.000000
19,161,339,مرقة ماجى دجاج شريط - 8 جم,2026-01-13,48.0,2026-01-15,6.916667


In [39]:
# =============================================================================
# Margin Boundaries Query - Get optimal, min, max boundaries from PRODUCT_STATISTICS
# =============================================================================
MARGIN_BOUNDARIES_QUERY = f'''
SELECT 
    region,
    product_id,
    optimal_bm,
    MIN_BOUNDARY,
    MAX_BOUNDARY,
    MEDIAN_BM
FROM (
    SELECT 
        region,
        product_id,
        target_bm,
        optimal_bm,
        MIN_BOUNDARY,
        MAX_BOUNDARY,
        MEDIAN_BM,
        MAX(created_at) OVER (PARTITION BY product_id, region) AS max_date,
        created_at
    FROM materialized_views.PRODUCT_STATISTICS
    WHERE created_at::DATE >= DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 60)
    QUALIFY max_date = created_at
)
'''

# Execute margin boundaries query
print("Loading margin boundaries data...")
df_margin_boundaries = query_snowflake(MARGIN_BOUNDARIES_QUERY)
df_margin_boundaries.columns = df_margin_boundaries.columns.str.lower()
df_margin_boundaries = convert_to_numeric(df_margin_boundaries)
print(f"Loaded {len(df_margin_boundaries)} margin boundary records")


Loading margin boundaries data...
Loaded 17918 margin boundary records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [42]:
# =============================================================================
# Add Margin Boundaries and Calculate Margin Tiers
# Tiers: 1 below min + 5 equally spaced in range + 2 above max = 8 tiers
# =============================================================================

# Merge margin boundaries with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_margin_boundaries[['region','product_id', 'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm']], 
    on=['product_id','region'],
    how='left'
)

# Calculate the effective minimum margin (min of MIN_BOUNDARY and optimal_bm)
pricing_with_discount['effective_min_margin'] = pricing_with_discount[['min_boundary', 'optimal_bm']].min(axis=1)

# Calculate step size: (max_boundary - effective_min_margin) / 4
# This gives 5 points including both endpoints
pricing_with_discount['margin_step'] = (
    pricing_with_discount['max_boundary'] - pricing_with_discount['effective_min_margin']
) / 4

# Calculate the 8 margin tiers:
# Tier -1: 1 step below minimum (below_min)
# Tiers 1-5: 5 equally spaced margins in [effective_min, max_boundary]
# Tier 6-7: 2 steps above maximum

# Below minimum (1 step below)
pricing_with_discount['margin_tier_below'] = pricing_with_discount['effective_min_margin'] - pricing_with_discount['margin_step']

# 5 tiers in range (equally spaced)
pricing_with_discount['margin_tier_1'] = pricing_with_discount['effective_min_margin']  # Min
pricing_with_discount['margin_tier_2'] = pricing_with_discount['effective_min_margin'] + pricing_with_discount['margin_step']
pricing_with_discount['margin_tier_3'] = pricing_with_discount['effective_min_margin'] + 2 * pricing_with_discount['margin_step']
pricing_with_discount['margin_tier_4'] = pricing_with_discount['effective_min_margin'] + 3 * pricing_with_discount['margin_step']
pricing_with_discount['margin_tier_5'] = pricing_with_discount['max_boundary']  # Max

# Above maximum (2 steps above)
pricing_with_discount['margin_tier_above_1'] = pricing_with_discount['max_boundary'] + pricing_with_discount['margin_step']
pricing_with_discount['margin_tier_above_2'] = pricing_with_discount['max_boundary'] + 2 * pricing_with_discount['margin_step']

# Fill NaN values for products without margin boundaries
margin_tier_cols = [
    'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
    'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2'
]

print(f"Margin boundaries and tiers added!")
print(f"\nRecords with margin boundaries: {len(pricing_with_discount[~pricing_with_discount['max_boundary'].isna()])}")
print(f"Records without margin boundaries: {len(pricing_with_discount[pricing_with_discount['max_boundary'].isna()])}")

print(f"\nMargin Tier Structure:")
print(f"  margin_tier_below:   effective_min - step (1 below)")
print(f"  margin_tier_1:       effective_min_margin")
print(f"  margin_tier_2:       effective_min + 1*step")
print(f"  margin_tier_3:       effective_min + 2*step")
print(f"  margin_tier_4:       effective_min + 3*step")
print(f"  margin_tier_5:       max_boundary")
print(f"  margin_tier_above_1: max_boundary + 1*step")
print(f"  margin_tier_above_2: max_boundary + 2*step")

print(f"\nSample margin tiers:")
pricing_with_discount[~pricing_with_discount['max_boundary'].isna()][
    ['product_id', 'sku', 'effective_min_margin', 'max_boundary', 'margin_step',
     'margin_tier_below', 'margin_tier_1', 'margin_tier_3', 'margin_tier_5', 
     'margin_tier_above_1', 'margin_tier_above_2']
].head(10)


Margin boundaries and tiers added!

Records with margin boundaries: 25032
Records without margin boundaries: 43348

Margin Tier Structure:
  margin_tier_below:   effective_min - step (1 below)
  margin_tier_1:       effective_min_margin
  margin_tier_2:       effective_min + 1*step
  margin_tier_3:       effective_min + 2*step
  margin_tier_4:       effective_min + 3*step
  margin_tier_5:       max_boundary
  margin_tier_above_1: max_boundary + 1*step
  margin_tier_above_2: max_boundary + 2*step

Sample margin tiers:


,product_id,sku,effective_min_margin,max_boundary,margin_step,margin_tier_below,margin_tier_1,margin_tier_3,margin_tier_5,margin_tier_above_1,margin_tier_above_2
0,11769,اوكسى يدوى لافندر - 330 جم,0.042000,0.081426,0.009857,0.032143,0.042000,0.061713,0.081426,0.091283,0.101139
1,12473,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,0.041076,0.062400,0.005331,0.035745,0.041076,0.051738,0.062400,0.067731,0.073062
2,19964,مولفيكس بريميوم حديث الولادة مقاس 1 - 58 حفاضة,0.030240,0.058200,0.006990,0.023250,0.030240,0.044220,0.058200,0.065190,0.072180
3,4966,اد-مي صلصة زجاج - 300 جم,0.031189,0.064489,0.008325,0.022865,0.031189,0.047839,0.064489,0.072814,0.081139
4,18964,كاتشب هاينز - 6 جرام,0.028548,0.058205,0.007414,0.021133,0.028548,0.043376,0.058205,0.065619,0.073034
5,23039,البوادي مربى كريمي تين - 365 جرام,0.060180,0.110004,0.012456,0.047724,0.060180,0.085092,0.110004,0.122460,0.134916
6,6494,سندة زيت خليط - 700 مل,0.040856,0.084755,0.010975,0.029881,0.040856,0.062805,0.084755,0.095730,0.106704
7,20486,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,0.064372,0.121066,0.014174,0.050198,0.064372,0.092719,0.121066,0.135240,0.149414
9,142,عصير جهينة كوكتيل - 235 مل,0.048477,0.070174,0.005424,0.043052,0.048477,0.059325,0.070174,0.075598,0.081023
10,74,حبوبة خل- 900 مل,0.132140,0.199122,0.016745,0.115395,0.132140,0.165631,0.199122,0.215867,0.232612


In [43]:
# =============================================================================
# Minimum Selling Quantity Query - Get min selling qty per product
# =============================================================================
MIN_SELLING_QTY_QUERY = f'''
SELECT product_id, min_selling_qty
FROM (
    SELECT *, MIN(basic_unit_count) OVER (PARTITION BY product_id) AS min_selling_qty
    FROM (
        SELECT DISTINCT
            pso.product_id,
            pso.PACKING_UNIT_ID,
            pup.basic_unit_count,
            SUM(pso.total_price) AS nmv,
            SUM(pso.total_price) / SUM(nmv) OVER (PARTITION BY pso.product_id) AS cntrb
        FROM product_sales_order pso
        JOIN PACKING_UNIT_PRODUCTS pup ON pup.product_id = pso.product_id 
            AND pup.PACKING_UNIT_ID = pso.PACKING_UNIT_ID
        JOIN sales_orders so ON so.id = pso.sales_order_id
        WHERE so.created_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120
            AND so.sales_order_status_id NOT IN (7, 12)
            AND so.channel IN ('telesales', 'retailer')
            AND pso.purchased_item_count <> 0
        GROUP BY ALL
        QUALIFY cntrb > 0.05
    )
    QUALIFY basic_unit_count = min_selling_qty
)
'''

# Execute min selling qty query
print("Loading minimum selling quantity data...")
df_min_selling_qty = query_snowflake(MIN_SELLING_QTY_QUERY)
df_min_selling_qty = convert_to_numeric(df_min_selling_qty)
print(f"Loaded {len(df_min_selling_qty)} min selling qty records")


Loading minimum selling quantity data...
Loaded 3884 min selling qty records


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


In [44]:
# =============================================================================
# Add Min Selling Qty and Below Min Stock Flag to pricing_with_discount
# =============================================================================

# Merge min selling qty with pricing_with_discount (by product_id)
pricing_with_discount = pricing_with_discount.merge(
    df_min_selling_qty[['product_id', 'min_selling_qty']], 
    on='product_id', 
    how='left'
)

# Fill missing min_selling_qty with 1 (default)
pricing_with_discount['min_selling_qty'] = pricing_with_discount['min_selling_qty'].fillna(1).astype(int)

# Create flag: below_min_stock_flag = 1 if (RR = 0 AND stocks > 0 AND stocks < min_selling_qty)
pricing_with_discount['below_min_stock_flag'] = np.where(
    (pricing_with_discount['in_stock_rr'] == 0) & 
    (pricing_with_discount['stocks'] > 0) &
    (pricing_with_discount['stocks'] < pricing_with_discount['min_selling_qty']),
    1, 0
)

print(f"Min selling qty and below_min_stock_flag added!")
print(f"\nSKUs flagged (zero RR & stocks < min_selling_qty): {len(pricing_with_discount[pricing_with_discount['below_min_stock_flag'] == 1])}")
print(f"SKUs not flagged: {len(pricing_with_discount[pricing_with_discount['below_min_stock_flag'] == 0])}")
print(f"\nSample flagged SKUs:")
pricing_with_discount[pricing_with_discount['below_min_stock_flag'] == 1][
    ['product_id', 'warehouse_id', 'sku', 'stocks', 'min_selling_qty', 'in_stock_rr', 'below_min_stock_flag']
].head(15)


Min selling qty and below_min_stock_flag added!

SKUs flagged (zero RR & stocks < min_selling_qty): 107
SKUs not flagged: 68273

Sample flagged SKUs:


,product_id,warehouse_id,sku,stocks,min_selling_qty,in_stock_rr,below_min_stock_flag
307,9122,337,مولبد ماكسى مضغوطة حماية ضد البكتيريا طويل جدا...,3,4,0.0,1
652,2709,632,تونة دولفين جولد فصوص بارد - 170 جم,4,15,0.0,1
1137,964,337,طحينة الرشيدى الميزان - 130 جم,2,10,0.0,1
2165,71,632,عسل البوادى اسود - 355 جم,4,6,0.0,1
2756,2709,236,تونة دولفين جولد فصوص بارد - 170 جم,11,15,0.0,1
3278,3557,797,تونة صن شاين اكسبريس قطع - 150 جم,3,12,0.0,1
3492,9877,797,تونة دولفين جولد فصوص حار - 170 جم,6,12,0.0,1
4239,633,339,توليدو تونة مفتتة حار سهلة الفتح- 140 جم,7,12,0.0,1
4364,3558,703,تونة صن شاين اكسبريس مفتتة بارد - 150 جم,4,12,0.0,1
4847,2215,797,اولويز الترا رفيع طويل جدا اقتصادى - 14 فوطة,3,4,0.0,1


In [45]:
# =============================================================================
# Yesterday's Discount Analysis Query
# Gets: SKU discount, Quantity discount, Tier 1/2/3 NMV breakdown and contributions
# =============================================================================
YESTERDAY_DISCOUNT_QUERY = f'''
WITH qd_det AS (
    -- Map dynamic tags to warehouse IDs using name matching
    SELECT DISTINCT 
        dt.id AS tag_id, 
        dt.name AS tag_name,
        REPLACE(w.name, ' ', '') AS warehouse_name,
        w.id AS warehouse_id,
        warehouse_name ILIKE '%' || CASE 
            WHEN SPLIT_PART(tag_name, '_', 1) = 'El' THEN SPLIT_PART(tag_name, '_', 2) 
            ELSE SPLIT_PART(tag_name, '_', 1) 
        END || '%' AS contains_flag
    FROM dynamic_tags dt
    JOIN dynamic_taggables dta ON dt.id = dta.dynamic_tag_id 
    CROSS JOIN warehouses w 
    WHERE dt.id > 3000
        AND dt.name LIKE '%QD_rets%'
        AND w.id IN (1, 236, 337, 8, 339, 170, 501, 401, 703, 632, 797, 962)
        AND contains_flag = 'true'
),

qd_config AS (
    SELECT * 
    FROM (
        SELECT 
            product_id,
            start_at,
            end_at,
            packing_unit_id,
            id AS qd_id,
            qd.warehouse_id,
            MAX(CASE WHEN tier = 1 THEN quantity END) AS tier_1_qty,
            MAX(CASE WHEN tier = 1 THEN discount_percentage END) AS tier_1_discount_pct,
            MAX(CASE WHEN tier = 2 THEN quantity END) AS tier_2_qty,
            MAX(CASE WHEN tier = 2 THEN discount_percentage END) AS tier_2_discount_pct,
            MAX(CASE WHEN tier = 3 THEN quantity END) AS tier_3_qty,
            MAX(CASE WHEN tier = 3 THEN discount_percentage END) AS tier_3_discount_pct
        FROM (
            SELECT 
                qd.id,
                qdv.product_id,
                qdv.packing_unit_id,
                qdv.quantity,
                qdv.discount_percentage,
                qd.dynamic_tag_id,
                qd.start_at,
                qd.end_at,
                ROW_NUMBER() OVER (
                    PARTITION BY qdv.product_id, qdv.packing_unit_id, qd.id 
                    ORDER BY qdv.quantity
                ) AS tier
            FROM quantity_discounts qd 
            JOIN quantity_discount_values qdv ON qd.id = qdv.quantity_discount_id 
            WHERE CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1 
                  BETWEEN qd.start_at::DATE AND qd.end_at::DATE
                AND qd.start_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 5
        ) qd_tiers
        JOIN qd_det qd ON qd.tag_id = qd_tiers.dynamic_tag_id
        GROUP BY ALL
    )
    QUALIFY ROW_NUMBER() OVER (PARTITION BY product_id, packing_unit_id, warehouse_id ORDER BY start_at DESC) = 1
),

-- Get all sales from yesterday
yesterday_sales AS (
    SELECT 
        pso.warehouse_id,
        pso.product_id,
        so.retailer_id,
        pso.packing_unit_id,
        pso.purchased_item_count AS qty,
        pso.total_price AS nmv,
        pso.item_price / pso.basic_unit_count AS unit_price,
        pso.ITEM_DISCOUNT_VALUE AS sku_discount_per_unit,
        pso.ITEM_QUANTITY_DISCOUNT_VALUE AS qty_discount_per_unit,
        pso.ITEM_DISCOUNT_VALUE * pso.purchased_item_count AS sku_discount_total,
        pso.ITEM_QUANTITY_DISCOUNT_VALUE * pso.purchased_item_count AS qty_discount_total,
        qd.tier_1_qty,
        qd.tier_2_qty,
        qd.tier_3_qty,
        qd.tier_1_discount_pct,
        qd.tier_2_discount_pct,
        qd.tier_3_discount_pct,
        -- Determine tier used
        CASE 
            WHEN pso.ITEM_QUANTITY_DISCOUNT_VALUE = 0 OR qd.tier_1_qty IS NULL THEN 'Base'
            WHEN qd.tier_3_qty IS NOT NULL AND pso.purchased_item_count >= qd.tier_3_qty THEN 'Tier 3'
            WHEN qd.tier_2_qty IS NOT NULL AND pso.purchased_item_count >= qd.tier_2_qty THEN 'Tier 2'
            WHEN qd.tier_1_qty IS NOT NULL AND pso.purchased_item_count >= qd.tier_1_qty THEN 'Tier 1'
            ELSE 'Base'
        END AS tier_used
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    LEFT JOIN qd_config qd 
        ON qd.product_id = pso.product_id 
        AND qd.packing_unit_id = pso.packing_unit_id
        AND qd.warehouse_id = so.warehouse_id
    WHERE so.created_at::DATE = CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
)

SELECT 
    warehouse_id,
    product_id,
    SUM(nmv) AS total_nmv,
    SUM(CASE WHEN sku_discount_per_unit > 0 THEN nmv ELSE 0 END) AS sku_discount_nmv,
    SUM(CASE WHEN qty_discount_per_unit > 0 THEN nmv ELSE 0 END) AS qty_discount_nmv,
    SUM(CASE WHEN tier_used = 'Tier 1' THEN nmv ELSE 0 END) AS tier1_nmv,
    SUM(CASE WHEN tier_used = 'Tier 2' THEN nmv ELSE 0 END) AS tier2_nmv,
    SUM(CASE WHEN tier_used = 'Tier 3' THEN nmv ELSE 0 END) AS tier3_nmv
FROM yesterday_sales
GROUP BY warehouse_id, product_id
HAVING SUM(nmv) > 0
ORDER BY total_nmv DESC
'''

# Execute yesterday discount query
print("Loading yesterday's discount analysis data...")
df_yesterday_discount = query_snowflake(YESTERDAY_DISCOUNT_QUERY)
df_yesterday_discount = convert_to_numeric(df_yesterday_discount)
print(f"Loaded {len(df_yesterday_discount)} SKU discount records from yesterday")

# Calculate contributions in Python
df_yesterday_discount['sku_discount_nmv_cntrb'] = (
    df_yesterday_discount['sku_discount_nmv'] / df_yesterday_discount['total_nmv'] * 100
).round(2)
df_yesterday_discount['qty_discount_nmv_cntrb'] = (
    df_yesterday_discount['qty_discount_nmv'] / df_yesterday_discount['total_nmv'] * 100
).round(2)
df_yesterday_discount['tier1_nmv_cntrb'] = (
    df_yesterday_discount['tier1_nmv'] / df_yesterday_discount['total_nmv'] * 100
).round(2)
df_yesterday_discount['tier2_nmv_cntrb'] = (
    df_yesterday_discount['tier2_nmv'] / df_yesterday_discount['total_nmv'] * 100
).round(2)
df_yesterday_discount['tier3_nmv_cntrb'] = (
    df_yesterday_discount['tier3_nmv'] / df_yesterday_discount['total_nmv'] * 100
).round(2)

# Summary
print(f"\n{'='*60}")
print(f"YESTERDAY'S DISCOUNT ANALYSIS SUMMARY")
print(f"{'='*60}")
print(f"\nTotal NMV yesterday: {df_yesterday_discount['total_nmv'].sum():,.0f}")
print(f"SKU Discount NMV: {df_yesterday_discount['sku_discount_nmv'].sum():,.0f}")
print(f"Quantity Discount NMV: {df_yesterday_discount['qty_discount_nmv'].sum():,.0f}")
print(f"\nNMV by Tier:")
print(f"  Tier 1: {df_yesterday_discount['tier1_nmv'].sum():,.0f}")
print(f"  Tier 2: {df_yesterday_discount['tier2_nmv'].sum():,.0f}")
print(f"  Tier 3: {df_yesterday_discount['tier3_nmv'].sum():,.0f}")

df_yesterday_discount.head(10)


Loading yesterday's discount analysis data...
Loaded 9090 SKU discount records from yesterday

YESTERDAY'S DISCOUNT ANALYSIS SUMMARY

Total NMV yesterday: 21,827,914
SKU Discount NMV: 4,746,944
Quantity Discount NMV: 3,361,950

NMV by Tier:
  Tier 1: 700,080
  Tier 2: 1,866,378
  Tier 3: 743,998


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


,warehouse_id,product_id,total_nmv,sku_discount_nmv,qty_discount_nmv,tier1_nmv,tier2_nmv,tier3_nmv,sku_discount_nmv_cntrb,qty_discount_nmv_cntrb,tier1_nmv_cntrb,tier2_nmv_cntrb,tier3_nmv_cntrb
0,1,589,337758.56,0.00,228936.00,3609.25,66241.25,159085.5,0.00,67.78,1.07,19.61,47.10
1,797,589,140522.67,40645.75,121696.25,2802.25,13402.50,105491.5,28.92,86.60,1.99,9.54,75.07
2,501,161,125926.50,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
3,236,62,119986.75,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
4,1,326,99220.00,43013.75,40342.00,14879.00,25463.00,0.0,43.35,40.66,15.00,25.66,0.00
5,236,151,98383.75,46429.25,0.00,0.00,0.00,0.0,47.19,0.00,0.00,0.00,0.00
6,236,8915,98331.31,60296.50,47271.75,2906.75,44365.00,0.0,61.32,48.07,2.96,45.12,0.00
7,337,589,92817.25,91257.75,0.00,0.00,0.00,0.0,98.32,0.00,0.00,0.00,0.00
8,236,326,91280.00,49289.75,32770.25,5629.00,27141.25,0.0,54.00,35.90,6.17,29.73,0.00
9,236,7885,90384.75,0.00,20040.00,6680.00,13360.00,0.0,0.00,22.17,7.39,14.78,0.00


In [46]:
# =============================================================================
# Add Yesterday's Discount Analysis to pricing_with_discount (Contributions Only)
# =============================================================================

# Merge yesterday discount data with pricing_with_discount - only contribution columns
pricing_with_discount = pricing_with_discount.merge(
    df_yesterday_discount[[
        'warehouse_id', 'product_id', 
        'sku_discount_nmv_cntrb', 'qty_discount_nmv_cntrb',
        'tier1_nmv_cntrb', 'tier2_nmv_cntrb', 'tier3_nmv_cntrb'
    ]].rename(columns={
        'sku_discount_nmv_cntrb': 'yesterday_sku_disc_cntrb',
        'qty_discount_nmv_cntrb': 'yesterday_qty_disc_cntrb',
        'tier1_nmv_cntrb': 'yesterday_t1_cntrb',
        'tier2_nmv_cntrb': 'yesterday_t2_cntrb',
        'tier3_nmv_cntrb': 'yesterday_t3_cntrb'
    }), 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill NaN for SKUs that had no sales yesterday
contrib_cols = [
    'yesterday_sku_disc_cntrb', 'yesterday_qty_disc_cntrb',
    'yesterday_t1_cntrb', 'yesterday_t2_cntrb', 'yesterday_t3_cntrb'
]
for col in contrib_cols:
    if col in pricing_with_discount.columns:
        pricing_with_discount[col] = pricing_with_discount[col].fillna(0)

print(f"Yesterday's discount contributions added!")
print(f"\nSKUs with discount data: {len(pricing_with_discount[pricing_with_discount['yesterday_sku_disc_cntrb'] > 0]) + len(pricing_with_discount[pricing_with_discount['yesterday_qty_disc_cntrb'] > 0])}")
print(f"\nSample data with yesterday's discount contributions:")
pricing_with_discount[
    ['product_id', 'warehouse_id', 'sku', 
     'yesterday_sku_disc_cntrb', 'yesterday_qty_disc_cntrb',
     'yesterday_t1_cntrb', 'yesterday_t2_cntrb', 'yesterday_t3_cntrb']
].head(15)


Yesterday's discount contributions added!

SKUs with discount data: 3011

Sample data with yesterday's discount contributions:


,product_id,warehouse_id,sku,yesterday_sku_disc_cntrb,yesterday_qty_disc_cntrb,yesterday_t1_cntrb,yesterday_t2_cntrb,yesterday_t3_cntrb
0,11769,337,اوكسى يدوى لافندر - 330 جم,0.00,0.0,0.0,0.0,0.0
1,12473,337,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,0.00,0.0,0.0,0.0,0.0
2,19964,501,مولفيكس بريميوم حديث الولادة مقاس 1 - 58 حفاضة,0.00,0.0,0.0,0.0,0.0
3,4966,337,اد-مي صلصة زجاج - 300 جم,0.00,0.0,0.0,0.0,0.0
4,18964,339,كاتشب هاينز - 6 جرام,0.00,0.0,0.0,0.0,0.0
5,23039,1,البوادي مربى كريمي تين - 365 جرام,60.00,0.0,0.0,0.0,0.0
6,6494,236,سندة زيت خليط - 700 مل,20.22,0.0,0.0,0.0,0.0
7,20486,337,اهلاوي بسكويت ساده كابيتيانو حجم جديد - 5 جنية,0.00,0.0,0.0,0.0,0.0
8,24122,632,لبان ترايدنت ليمون و توت 7 قطع - طعم جديد 7 قطعه,0.00,0.0,0.0,0.0,0.0
9,142,339,عصير جهينة كوكتيل - 235 مل,0.00,0.0,0.0,0.0,0.0


In [47]:
# =============================================================================
# Performance Benchmark Query
# Gets: Yesterday qty, Recent 7d qty, MTD qty, and P80 benchmarks (240 days)
# Uses materialized_views.stock_day_close for in-stock determination
# =============================================================================
PERFORMANCE_BENCHMARK_QUERY = f'''
WITH params AS (
    SELECT
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS today,
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 1 AS yesterday,
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 240 AS history_start,
        DATE_TRUNC('month', CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE) AS current_month_start,
        DAY(CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE) AS current_day_of_month
),

-- Daily sales aggregation (240 days)
daily_sales AS (
    SELECT
        pso.warehouse_id,
        pso.product_id,
        so.created_at::DATE AS sale_date,
        SUM(pso.purchased_item_count * pso.basic_unit_count) AS daily_qty
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    CROSS JOIN params p
    WHERE so.created_at::DATE >= p.history_start
        AND so.created_at::DATE < p.today
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY pso.warehouse_id, pso.product_id, so.created_at::DATE
),

-- Daily stock status using stock_day_close
-- In-stock = opening (prev day close) > 0 AND closing > 0
daily_stock AS (
    SELECT
        sdc.warehouse_id,
        sdc.product_id,
        sdc.TIMESTAMP::DATE AS stock_date,
        sdc.available_stock,
        LAG(sdc.available_stock, 1) OVER (
            PARTITION BY sdc.warehouse_id, sdc.product_id 
            ORDER BY sdc.TIMESTAMP::DATE
        ) AS opening_stock,
        CASE 
            WHEN LAG(sdc.available_stock, 1) OVER (
                    PARTITION BY sdc.warehouse_id, sdc.product_id ORDER BY sdc.TIMESTAMP::DATE
                 ) > 0 
                 AND sdc.available_stock > 0 
            THEN 1 
            ELSE 0 
        END AS in_stock_flag
    FROM materialized_views.stock_day_close sdc
    CROSS JOIN params p
    WHERE sdc.TIMESTAMP::DATE >= p.history_start - 1  -- Need one extra day for LAG
        AND sdc.TIMESTAMP::DATE < p.today
),

-- Combine sales with stock status
daily_with_stock AS (
    SELECT
        COALESCE(ds.warehouse_id, st.warehouse_id) AS warehouse_id,
        COALESCE(ds.product_id, st.product_id) AS product_id,
        COALESCE(ds.sale_date, st.stock_date) AS the_date,
        COALESCE(ds.daily_qty, 0) AS daily_qty,
        COALESCE(st.in_stock_flag, 0) AS in_stock_flag
    FROM daily_sales ds
    FULL OUTER JOIN daily_stock st 
        ON ds.warehouse_id = st.warehouse_id 
        AND ds.product_id = st.product_id 
        AND ds.sale_date = st.stock_date
    WHERE COALESCE(ds.sale_date, st.stock_date) >= (SELECT history_start FROM params)
),

-- Calculate P80 benchmark (in-stock days only, 240 days, EXCLUDING last 7 days)
p80_daily_benchmark AS (
    SELECT
        warehouse_id,
        product_id,
        PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY daily_qty) AS p80_daily_240d,
        AVG(daily_qty) AS avg_daily_240d,
        STDDEV(daily_qty) AS std_daily_240d,
        COUNT(*) AS in_stock_days_240d
    FROM daily_with_stock
    CROSS JOIN params p
    WHERE in_stock_flag = 1
        AND the_date >= p.history_start
        AND the_date < p.today - 7  -- Exclude last 7 days from benchmark
    GROUP BY warehouse_id, product_id
),

-- Calculate 7-day rolling SUM for P80 recent benchmark
rolling_7d AS (
    SELECT
        warehouse_id,
        product_id,
        the_date,
        SUM(daily_qty) OVER (
            PARTITION BY warehouse_id, product_id 
            ORDER BY the_date 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7d_sum,
        SUM(in_stock_flag) OVER (
            PARTITION BY warehouse_id, product_id 
            ORDER BY the_date 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS in_stock_days_7d
    FROM daily_with_stock
),

p80_7d_benchmark AS (
    SELECT
        warehouse_id,
        product_id,
        PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY rolling_7d_sum) AS p80_7d_rolling_240d
    FROM rolling_7d
    CROSS JOIN params p
    WHERE the_date >= p.history_start + 7  -- Need 7 days for rolling
        AND the_date < p.today - 7  -- Exclude last 7 days from benchmark
        AND in_stock_days_7d >= 4  -- At least 4 of 7 days in stock
    GROUP BY warehouse_id, product_id
),

-- MTD benchmark: P80 of same MTD period totals (last 12 months)
-- Sum all sales from day 1 to current day of month for each historical month
mtd_historical AS (
    SELECT
        dws.warehouse_id,
        dws.product_id,
        DATE_TRUNC('month', dws.the_date) AS period_month_start,
        SUM(dws.daily_qty) AS mtd_total_qty  -- Sum of all days from 1 to current_day_of_month
    FROM daily_with_stock dws
    CROSS JOIN params p
    WHERE DAY(dws.the_date) <= p.current_day_of_month  -- Only days up to current day of month
    GROUP BY dws.warehouse_id, dws.product_id, DATE_TRUNC('month', dws.the_date)
),

mtd_by_period AS (
    SELECT
        mh.warehouse_id,
        mh.product_id,
        mh.period_month_start,
        mh.mtd_total_qty AS mtd_qty_at_day  -- Total MTD qty for that month
    FROM mtd_historical mh
    CROSS JOIN params p
    WHERE mh.period_month_start >= DATEADD(month, -12, p.current_month_start)
        AND mh.period_month_start < p.current_month_start
),

p80_mtd_benchmark AS (
    SELECT
        warehouse_id,
        product_id,
        PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY mtd_qty_at_day) AS p80_mtd_12mo,
        AVG(mtd_qty_at_day) AS avg_mtd_12mo
    FROM mtd_by_period
    GROUP BY warehouse_id, product_id
    HAVING COUNT(*) >= 3  -- At least 3 months of data
),

-- Current period quantities
current_metrics AS (
    SELECT
        warehouse_id,
        product_id,
        -- Yesterday
        SUM(CASE WHEN the_date = (SELECT yesterday FROM params) THEN daily_qty ELSE 0 END) AS yesterday_qty,
        -- Recent 7 days
        SUM(CASE WHEN the_date >= (SELECT today FROM params) - 7 AND the_date < (SELECT today FROM params) THEN daily_qty ELSE 0 END) AS recent_7d_qty,
        SUM(CASE WHEN the_date >= (SELECT today FROM params) - 7 AND the_date < (SELECT today FROM params) AND in_stock_flag = 1 THEN 1 ELSE 0 END) AS recent_7d_in_stock_days,
        -- MTD
        SUM(CASE WHEN the_date >= (SELECT current_month_start FROM params) AND the_date < (SELECT today FROM params) THEN daily_qty ELSE 0 END) AS mtd_qty
    FROM daily_with_stock
    GROUP BY warehouse_id, product_id
)

-- Final output
SELECT
    cm.warehouse_id,
    cm.product_id,
    
    -- Current period quantities
    cm.yesterday_qty,
    cm.recent_7d_qty,
    cm.recent_7d_in_stock_days,
    cm.mtd_qty,
    
    -- Benchmarks
    COALESCE(pb.p80_daily_240d, 1) AS p80_daily_240d,
    COALESCE(pb.avg_daily_240d, 0) AS avg_daily_240d,
    COALESCE(pb.in_stock_days_240d, 0) AS in_stock_days_240d,
    COALESCE(p7.p80_7d_rolling_240d, pb.p80_daily_240d * 7, 1) AS p80_7d_sum_240d,
    COALESCE(pm.p80_mtd_12mo, pb.p80_daily_240d * (SELECT current_day_of_month FROM params), 1) AS p80_mtd_12mo,
    
    -- Performance ratios (all comparing sums to sums)
    -- Yesterday: daily qty vs P80 daily
    ROUND(cm.yesterday_qty / NULLIF(COALESCE(pb.p80_daily_240d, 1), 0), 2) AS yesterday_ratio,
    -- Recent 7d: 7-day sum vs P80 of 7-day sums
    ROUND(cm.recent_7d_qty / NULLIF(COALESCE(p7.p80_7d_rolling_240d, pb.p80_daily_240d * 7, 1), 0), 2) AS recent_ratio,
    -- MTD: MTD sum vs P80 of historical MTD sums
    ROUND(cm.mtd_qty / NULLIF(COALESCE(pm.p80_mtd_12mo, pb.p80_daily_240d * (SELECT current_day_of_month FROM params), 1), 0), 2) AS mtd_ratio

FROM current_metrics cm
LEFT JOIN p80_daily_benchmark pb ON cm.warehouse_id = pb.warehouse_id AND cm.product_id = pb.product_id
LEFT JOIN p80_7d_benchmark p7 ON cm.warehouse_id = p7.warehouse_id AND cm.product_id = p7.product_id
LEFT JOIN p80_mtd_benchmark pm ON cm.warehouse_id = pm.warehouse_id AND cm.product_id = pm.product_id
where cm.warehouse_id in (1, 236, 337, 8, 339, 170, 501, 401, 703, 632, 797, 962)
'''

# Execute benchmark query
print("Loading performance benchmark data (this may take a moment due to 240-day history)...")
df_benchmarks = query_snowflake(PERFORMANCE_BENCHMARK_QUERY)
df_benchmarks = convert_to_numeric(df_benchmarks)
print(f"Loaded {len(df_benchmarks)} benchmark records")

# Preview
df_benchmarks.head(10)


Loading performance benchmark data (this may take a moment due to 240-day history)...


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


Loaded 293725 benchmark records


,warehouse_id,product_id,yesterday_qty,recent_7d_qty,recent_7d_in_stock_days,mtd_qty,p80_daily_240d,avg_daily_240d,in_stock_days_240d,p80_7d_sum_240d,p80_mtd_12mo,yesterday_ratio,recent_ratio,mtd_ratio
0,337,23251,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
1,703,20030,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
2,703,19724,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
3,501,5182,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
4,797,7232,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
5,8,15896,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
6,339,25251,0,0,0,0,1.0,0.0,0,1.0,1.0,0.0,0.0,0.0
7,170,22095,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN
8,501,9379,1,3,7,3,0.0,0.0,142,0.0,0.0,NaN,NaN,NaN
9,339,103,0,0,0,0,1.0,0.0,0,1.0,0.0,0.0,0.0,NaN


In [48]:
# =============================================================================
# Add Performance Benchmarks and Tags to pricing_with_discount
# =============================================================================

# Merge benchmark data with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_benchmarks[[
        'warehouse_id', 'product_id',
        'yesterday_qty', 'recent_7d_qty', 'recent_7d_in_stock_days', 'mtd_qty',
        'p80_daily_240d', 'avg_daily_240d', 'in_stock_days_240d',
        'p80_7d_sum_240d', 'p80_mtd_12mo',
        'yesterday_ratio', 'recent_ratio', 'mtd_ratio'
    ]], 
    on=['warehouse_id', 'product_id'], 
    how='left'
)

# Fill NaN values
qty_cols = ['yesterday_qty', 'recent_7d_qty', 'recent_7d_in_stock_days', 'mtd_qty']
for col in qty_cols:
    pricing_with_discount[col] = pricing_with_discount[col].fillna(0)

benchmark_cols = ['p80_daily_240d', 'p80_7d_sum_240d', 'p80_mtd_12mo']
for col in benchmark_cols:
    pricing_with_discount[col] = pricing_with_discount[col].fillna(1)  # Default to 1 to avoid division issues

ratio_cols = ['yesterday_ratio', 'recent_ratio', 'mtd_ratio']
for col in ratio_cols:
    pricing_with_discount[col] = pricing_with_discount[col].fillna(0)

pricing_with_discount['avg_daily_240d'] = pricing_with_discount['avg_daily_240d'].fillna(0)
pricing_with_discount['in_stock_days_240d'] = pricing_with_discount['in_stock_days_240d'].fillna(0)

# =============================================================================
# Performance Tags - Classify each ratio
# =============================================================================
def get_performance_tag(ratio):
    """Classify performance based on ratio to benchmark"""
    if pd.isna(ratio) or ratio == 0:
        return 'No Data'
    elif ratio >= 2.0:
        return 'Star Performer'      # 🌟 100%+ above benchmark
    elif ratio >= 1.5:
        return 'Over Achiever'       # 🔥 50%+ above benchmark  
    elif ratio >= 1.0:
        return 'On Track'            # ✅ Meeting benchmark
    elif ratio >= 0.7:
        return 'Underperforming'     # ⚠️ Below benchmark
    elif ratio >= 0.4:
        return 'Struggling'          # 🔻 Significantly below
    else:
        return 'Critical'            # 🚨 Needs intervention

# Apply tags to each timeframe
pricing_with_discount['yesterday_status'] = pricing_with_discount['yesterday_ratio'].apply(get_performance_tag)
pricing_with_discount['recent_status'] = pricing_with_discount['recent_ratio'].apply(get_performance_tag)
pricing_with_discount['mtd_status'] = pricing_with_discount['mtd_ratio'].apply(get_performance_tag)

# =============================================================================
# Combined Performance Score (weighted average of ratios)
# =============================================================================
# Weight: Yesterday 20%, Recent 7d 40%, MTD 40%
pricing_with_discount['combined_perf_ratio'] = (
    0.2 * pricing_with_discount['yesterday_ratio'].clip(upper=3) +  # Cap at 3x to limit outlier impact
    0.4 * pricing_with_discount['recent_ratio'].clip(upper=3) +
    0.4 * pricing_with_discount['mtd_ratio'].clip(upper=3)
)

pricing_with_discount['combined_status'] = pricing_with_discount['combined_perf_ratio'].apply(get_performance_tag)

# =============================================================================
# High Performer Flag (for immediate action consideration)
# =============================================================================
# Flag SKUs that are significantly over-achieving and may need action (price increase, etc.)
pricing_with_discount['high_performer_flag'] = np.where(
    (pricing_with_discount['yesterday_ratio'] >= 1.5) & 
    (pricing_with_discount['recent_ratio'] >= 1.3) &
    (pricing_with_discount['mtd_ratio'] >= 1.2),
    1, 0
)

# Star performer flag (exceptional - all metrics 2x+ benchmark)
pricing_with_discount['star_performer_flag'] = np.where(
    (pricing_with_discount['yesterday_ratio'] >= 2.0) & 
    (pricing_with_discount['recent_ratio'] >= 1.5) &
    (pricing_with_discount['mtd_ratio'] >= 1.5),
    1, 0
)

# =============================================================================
# Summary
# =============================================================================
print(f"Performance benchmarks added!")
print(f"\n{'='*60}")
print(f"PERFORMANCE STATUS DISTRIBUTION")
print(f"{'='*60}")

print(f"\nYesterday Status:")
print(pricing_with_discount['yesterday_status'].value_counts().to_string())

print(f"\nRecent 7d Status:")
print(pricing_with_discount['recent_status'].value_counts().to_string())

print(f"\nMTD Status:")
print(pricing_with_discount['mtd_status'].value_counts().to_string())

print(f"\nCombined Status:")
print(pricing_with_discount['combined_status'].value_counts().to_string())

print(f"\n{'='*60}")
print(f"HIGH PERFORMERS (Action Candidates)")
print(f"{'='*60}")
print(f"High Performers (flag=1): {len(pricing_with_discount[pricing_with_discount['high_performer_flag'] == 1])}")
print(f"Star Performers (flag=1): {len(pricing_with_discount[pricing_with_discount['star_performer_flag'] == 1])}")

# Show top performers
print(f"\nTop 15 Star Performers:")
pricing_with_discount[pricing_with_discount['star_performer_flag'] == 1].nlargest(15, 'combined_perf_ratio')[
    ['product_id', 'warehouse_id', 'sku', 
     'yesterday_ratio', 'recent_ratio', 'mtd_ratio', 'combined_perf_ratio',
     'yesterday_status', 'combined_status']
]


Performance benchmarks added!

PERFORMANCE STATUS DISTRIBUTION

Yesterday Status:
yesterday_status
No Data            62272
Critical            1568
Struggling          1374
On Track            1331
Star Performer      1001
Underperforming      516
Over Achiever        318

Recent 7d Status:
recent_status
No Data            56343
Critical            3374
Struggling          3197
On Track            2159
Underperforming     1742
Star Performer       934
Over Achiever        631

MTD Status:
mtd_status
No Data            54863
Struggling          3207
Critical            3069
Underperforming     2439
On Track            2430
Star Performer      1477
Over Achiever        895

Combined Status:
combined_status
No Data            54192
Critical            4506
Struggling          3759
Underperforming     2362
On Track            2157
Over Achiever        849
Star Performer       555

HIGH PERFORMERS (Action Candidates)
High Performers (flag=1): 367
Star Performers (flag=1): 234

Top 15 Star 

,product_id,warehouse_id,sku,yesterday_ratio,recent_ratio,mtd_ratio,combined_perf_ratio,yesterday_status,combined_status
541,1234,236,سمن جنة ظرف - 350 جم,7.50,3.09,14.58,3.0,Star Performer,Star Performer
4691,19687,703,اريال أتوماتيك داونى - 9 كجم,4.00,5.17,3.53,3.0,Star Performer,Star Performer
11332,22087,1,جليد مالتي سبراي سحر الشرق - 460 مل,20.50,3.21,3.75,3.0,Star Performer,Star Performer
13476,25288,337,بسكويت شاي شوجر بجوز الهند- 5 جنية,10.00,13.00,13.00,3.0,Star Performer,Star Performer
13477,25288,337,بسكويت شاي شوجر بجوز الهند- 5 جنية,10.00,13.00,13.00,3.0,Star Performer,Star Performer
13478,25288,337,بسكويت شاي شوجر بجوز الهند- 5 جنية,10.00,13.00,13.00,3.0,Star Performer,Star Performer
17826,25286,1,.حبوبه سمن نباتي صفرا - 700 جم,7.00,8.00,8.00,3.0,Star Performer,Star Performer
19794,7573,339,كلوروكس الوان باودر جديد 5 جنيه - 30 جم,5.36,5.67,7.02,3.0,Star Performer,Star Performer
21892,24618,337,سميليز شوكو ميكس - 24 قطعه,3.00,3.00,3.00,3.0,Star Performer,Star Performer
22578,13486,236,مكرونة شعرية بساطة - 1 كجم,5.50,3.50,3.83,3.0,Star Performer,Star Performer


In [49]:
# =============================================================================
# No NMV in Last 4 Months Flag
# Identifies SKUs that have not generated any NMV in the past 4 months (120 days)
# =============================================================================
NO_NMV_4M_QUERY = f'''
WITH nmv_last_4m AS (
    SELECT 
        pso.warehouse_id,
        pso.product_id,
        SUM(pso.total_price) AS total_nmv_4m
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    WHERE so.created_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120
        AND so.created_at::DATE < CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY pso.warehouse_id, pso.product_id
    HAVING SUM(pso.total_price) > 0
)
SELECT 
    warehouse_id,
    product_id,
    total_nmv_4m
FROM nmv_last_4m
'''

# Execute query
print("Loading SKUs with NMV in last 4 months...")
df_nmv_4m = query_snowflake(NO_NMV_4M_QUERY)
df_nmv_4m = convert_to_numeric(df_nmv_4m)
print(f"Found {len(df_nmv_4m)} SKU-warehouse combinations with NMV in last 4 months")

# Merge and create no_nmv_4m flag
pricing_with_discount = pricing_with_discount.merge(
    df_nmv_4m[['warehouse_id', 'product_id', 'total_nmv_4m']],
    on=['warehouse_id', 'product_id'],
    how='left'
)

# Flag SKUs with no NMV in last 4 months
# 1 = No NMV (should potentially be filtered), 0 = Has NMV
pricing_with_discount['no_nmv_4m'] = np.where(
    pricing_with_discount['total_nmv_4m'].isna() | (pricing_with_discount['total_nmv_4m'] == 0),
    1, 0
)

# Fill NaN for total_nmv_4m
pricing_with_discount['total_nmv_4m'] = pricing_with_discount['total_nmv_4m'].fillna(0)

print(f"\n{'='*60}")
print(f"NO NMV IN LAST 4 MONTHS ANALYSIS")
print(f"{'='*60}")
print(f"Total records: {len(pricing_with_discount)}")
print(f"SKUs with NO NMV in 4 months (no_nmv_4m=1): {len(pricing_with_discount[pricing_with_discount['no_nmv_4m'] == 1])}")
print(f"SKUs with NMV in 4 months (no_nmv_4m=0): {len(pricing_with_discount[pricing_with_discount['no_nmv_4m'] == 0])}")

# Show sample of SKUs with no NMV
print(f"\nSample SKUs with no NMV in last 4 months:")
pricing_with_discount[pricing_with_discount['no_nmv_4m'] == 1][
    ['product_id', 'warehouse_id', 'sku', 'stocks', 'in_stock_rr', 'zero_demand', 'no_nmv_4m']
].head(15)


Loading SKUs with NMV in last 4 months...
Found 28612 SKU-warehouse combinations with NMV in last 4 months

NO NMV IN LAST 4 MONTHS ANALYSIS
Total records: 68380
SKUs with NO NMV in 4 months (no_nmv_4m=1): 48056
SKUs with NMV in 4 months (no_nmv_4m=0): 20324

Sample SKUs with no NMV in last 4 months:


/tmp/ipykernel_25983/3044313561.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


,product_id,warehouse_id,sku,stocks,in_stock_rr,zero_demand,no_nmv_4m
1,12473,337,مولبد الترا رفيعة طويل 6 + 1 فوطة مجانا - 7 فوطة,0,0.0,0,1
8,24122,632,لبان ترايدنت ليمون و توت 7 قطع - طعم جديد 7 قطعه,0,0.0,0,1
11,11999,632,الضحى مكرونة بصوص الطماطم الإيطالي - 185 جم,0,0.0,0,1
12,11215,501,باربكيو هاينز دوى باك - 125 جم,0,0.0,0,1
15,1512,401,حلو الشام كيك فانيليا- 400 جم,0,0.0,0,1
16,11,501,سمن كريستال ابيض ظرف - 55 جم,0,0.0,0,1
21,313,797,كامي صابون اناقة- 170 جم,0,0.0,0,1
22,286,703,كريم شانتيه دريم ظرف - 45 جم,0,0.0,0,1
24,2736,632,شوكولاتة كادبورى بندق - 50 جم,0,0.0,0,1
32,170,337,كوكس كاكاو- 40 جم,0,0.0,0,1


In [ ]:
# =============================================================================
# Normal Refill Query - Avg qty & stddev for frequent retailers (last 120 days)
# Frequent retailer definition based on ABC classification (from existing dataframe):
#   - Class A: bought 4+ times
#   - Class B: bought 3+ times
#   - Class C: bought 2+ times
# =============================================================================
NORMAL_REFILL_QUERY = f'''
WITH params AS (
    SELECT 
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS today,
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE - 120 AS history_start
),

-- Get retailer order counts per product-warehouse (last 120 days)
retailer_orders AS (
    SELECT 
        pso.warehouse_id,
        pso.product_id,
        so.retailer_id,
        COUNT(DISTINCT so.id) AS order_count
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    CROSS JOIN params p
    WHERE so.created_at::DATE >= p.history_start
        AND so.created_at::DATE < p.today
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY pso.warehouse_id, pso.product_id, so.retailer_id
),

-- Get individual order quantities per retailer
order_quantities AS (
    SELECT 
        pso.warehouse_id,
        pso.product_id,
        so.retailer_id,
        so.id AS order_id,
        SUM(pso.purchased_item_count * pso.basic_unit_count) AS order_qty
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    CROSS JOIN params p
    WHERE so.created_at::DATE >= p.history_start
        AND so.created_at::DATE < p.today
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
    GROUP BY pso.warehouse_id, pso.product_id, so.retailer_id, so.id
)

-- Return retailer-level data with order counts for Python filtering
SELECT 
    oq.warehouse_id,
    oq.product_id,
    oq.retailer_id,
    ro.order_count,
    oq.order_id,
    oq.order_qty
FROM order_quantities oq
JOIN retailer_orders ro 
    ON ro.warehouse_id = oq.warehouse_id 
    AND ro.product_id = oq.product_id 
    AND ro.retailer_id = oq.retailer_id
'''

# Execute normal refill query
print("Loading retailer order data for normal refill calculation (last 120 days)...")
df_retailer_orders = query_snowflake(NORMAL_REFILL_QUERY)
df_retailer_orders = convert_to_numeric(df_retailer_orders)
print(f"Loaded {len(df_retailer_orders)} retailer order records")

# Get ABC classification from existing dataframe
abc_mapping = pricing_with_discount[['warehouse_id', 'product_id', 'abc_class']].drop_duplicates()
print(f"ABC classification mapping: {len(abc_mapping)} product-warehouse combinations")

# Merge ABC classification into retailer orders
df_retailer_orders = df_retailer_orders.merge(
    abc_mapping,
    on=['warehouse_id', 'product_id'],
    how='inner'
)
print(f"Records after ABC merge: {len(df_retailer_orders)}")

# Filter frequent retailers based on ABC class thresholds
# Class A: 4+ orders, Class B: 3+ orders, Class C: 2+ orders
df_frequent = df_retailer_orders[
    ((df_retailer_orders['abc_class'] == 'A') & (df_retailer_orders['order_count'] >= 4)) |
    ((df_retailer_orders['abc_class'] == 'B') & (df_retailer_orders['order_count'] >= 3)) |
    ((df_retailer_orders['abc_class'] == 'C') & (df_retailer_orders['order_count'] >= 2))
].copy()
print(f"Records from frequent retailers: {len(df_frequent)}")

# Calculate normal_refill (avg qty) and refill_stddev per product-warehouse
df_normal_refill = df_frequent.groupby(['warehouse_id', 'product_id']).agg(
    frequent_retailer_count=('retailer_id', 'nunique'),
    frequent_order_count=('order_id', 'nunique'),
    normal_refill=('order_qty', 'mean'),
    refill_stddev=('order_qty', 'std')
).reset_index()

# Round values and fill NaN stddev (when only 1 order)
df_normal_refill['normal_refill'] = df_normal_refill['normal_refill'].round(2)
df_normal_refill['refill_stddev'] = df_normal_refill['refill_stddev'].fillna(0).round(2)

# Filter to products with at least 2 orders for meaningful stats
df_normal_refill = df_normal_refill[df_normal_refill['frequent_order_count'] >= 2]
print(f"Final normal refill records (min 2 orders): {len(df_normal_refill)}")

# Merge with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_normal_refill[['warehouse_id', 'product_id', 'frequent_retailer_count', 
                      'frequent_order_count', 'normal_refill', 'refill_stddev']],
    on=['warehouse_id', 'product_id'],
    how='left'
)

# Fill NaN values
pricing_with_discount['frequent_retailer_count'] = pricing_with_discount['frequent_retailer_count'].fillna(0)
pricing_with_discount['frequent_order_count'] = pricing_with_discount['frequent_order_count'].fillna(0)
pricing_with_discount['normal_refill'] = pricing_with_discount['normal_refill'].fillna(0)
pricing_with_discount['refill_stddev'] = pricing_with_discount['refill_stddev'].fillna(0)

print(f"\n{'='*60}")
print(f"NORMAL REFILL ANALYSIS (Frequent Retailers - 120 days)")
print(f"{'='*60}")
print(f"Records with normal_refill data: {len(pricing_with_discount[pricing_with_discount['normal_refill'] > 0])}")
print(f"Records without normal_refill data: {len(pricing_with_discount[pricing_with_discount['normal_refill'] == 0])}")
print(f"\nNormal refill distribution:")
print(pricing_with_discount[pricing_with_discount['normal_refill'] > 0]['normal_refill'].describe())
print(f"\nSample data:")
pricing_with_discount[pricing_with_discount['normal_refill'] > 0][
    ['product_id', 'warehouse_id', 'sku', 'abc_class', 'frequent_retailer_count', 
     'frequent_order_count', 'normal_refill', 'refill_stddev', 'in_stock_rr']
].head(15)


In [ ]:
# =============================================================================
# Live Cart Rules Query - Get current cart rules from the system
# Merges on product_id and cohort_id
# =============================================================================
LIVE_CART_RULES_QUERY = f'''
SELECT 
    cppu.cohort_id,
    pup.product_id,
    pup.packing_unit_id,
    pup.basic_unit_count,
    COALESCE(cppu.MAX_PER_SALES_ORDER, cppu2.MAX_PER_SALES_ORDER) AS current_cart_rule
FROM COHORT_PRODUCT_PACKING_UNITS cppu 
JOIN PACKING_UNIT_PRODUCTS pup ON cppu.PRODUCT_PACKING_UNIT_ID = pup.id 
JOIN cohorts c ON c.id = cppu.cohort_id
LEFT JOIN COHORT_PRODUCT_PACKING_UNITS cppu2 
    ON cppu.PRODUCT_PACKING_UNIT_ID = cppu2.PRODUCT_PACKING_UNIT_ID 
    AND cppu2.cohort_id = c.FALLBACK_COHORT_ID
WHERE cppu.cohort_id IN ({','.join(map(str, COHORT_IDS))})
'''

# Execute live cart rules query
print("Loading live cart rules...")
df_cart_rules = query_snowflake(LIVE_CART_RULES_QUERY)
df_cart_rules = convert_to_numeric(df_cart_rules)
print(f"Loaded {len(df_cart_rules)} cart rule records")

# Aggregate to product-cohort level (take the cart rule for basic unit, or min if multiple)
# Filter to basic unit (packing_unit_id where basic_unit_count = 1) for simpler merging
df_cart_rules_basic = df_cart_rules[df_cart_rules['basic_unit_count'] == 1].copy()
print(f"Basic unit cart rules: {len(df_cart_rules_basic)}")

# If no basic unit, take the minimum cart rule per product-cohort
df_cart_rules_agg = df_cart_rules.groupby(['cohort_id', 'product_id']).agg(
    current_cart_rule=('current_cart_rule', 'min')
).reset_index()

# Prefer basic unit cart rule, fallback to aggregated
df_cart_rules_final = df_cart_rules_basic[['cohort_id', 'product_id', 'current_cart_rule']].drop_duplicates()
df_cart_rules_final = df_cart_rules_final.merge(
    df_cart_rules_agg[['cohort_id', 'product_id', 'current_cart_rule']].rename(columns={'current_cart_rule': 'cart_rule_agg'}),
    on=['cohort_id', 'product_id'],
    how='outer'
)
df_cart_rules_final['current_cart_rule'] = df_cart_rules_final['current_cart_rule'].fillna(df_cart_rules_final['cart_rule_agg'])
df_cart_rules_final = df_cart_rules_final[['cohort_id', 'product_id', 'current_cart_rule']].drop_duplicates()
print(f"Final cart rules (product-cohort level): {len(df_cart_rules_final)}")

# Merge with pricing_with_discount
pricing_with_discount = pricing_with_discount.merge(
    df_cart_rules_final,
    on=['cohort_id', 'product_id'],
    how='left'
)

# Fill NaN cart rules with 0 (no cart rule set)
pricing_with_discount['current_cart_rule'] = pricing_with_discount['current_cart_rule'].fillna(0)

print(f"\n{'='*60}")
print(f"LIVE CART RULES ANALYSIS")
print(f"{'='*60}")
print(f"Records with cart rule > 0: {len(pricing_with_discount[pricing_with_discount['current_cart_rule'] > 0])}")
print(f"Records without cart rule: {len(pricing_with_discount[pricing_with_discount['current_cart_rule'] == 0])}")
print(f"\nCart rule distribution:")
print(pricing_with_discount[pricing_with_discount['current_cart_rule'] > 0]['current_cart_rule'].describe())
print(f"\nSample data with cart rules:")
pricing_with_discount[pricing_with_discount['current_cart_rule'] > 0][
    ['product_id', 'cohort_id', 'warehouse_id', 'sku', 'current_price', 'current_cart_rule', 'in_stock_rr']
].head(15)


In [51]:
final_df = pricing_with_discount[(pricing_with_discount['no_nmv_4m']==0)|(pricing_with_discount['stocks']>0)]